In [ ]:
#installing the text extraction module
!pip install pdfminer.six

     |████████████████████████████████| 5.6 MB 3.9 MB/s 
     |████████████████████████████████| 3.2 MB 36.2 MB/s 


In [ ]:

#loading libraries
import numpy as np
import os
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer 
from pdfminer.high_level import extract_text
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#downloading extra data for the deployed NLP library (nltk)
nltk.download('stopwords')
nltk.download('punkt')
stopword = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
ps = PorterStemmer()  #for stemming

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:
#function for extracting mda text files
def mda_text_extractor(file_name,start_page, end_page):
  text = extract_text(file_name,page_numbers=list(np.arange(start_page,end_page+1)))
  return text

#function for writing mda text files into a text file
def write_to_text(textf, name):
  os.makedirs(os.path.dirname(f'MD&A_text_files/{name}'), exist_ok=True)
  with open(f'MD&A_text_files/{name}', 'w',encoding='utf-8') as ofile:
    ofile.write(textf)

#function for preprocessing text 
def prepare(name):  
  mylines = []                            
  with open (f'MD&A_text_files/{name}', 'rt',encoding='utf-8', errors='ignore') as myfile:
      for line in myfile:
          line1 = re.sub('[^a-zA-Z]',' ', line) #removing all except words
          line2 = line1.rstrip().lstrip().lower() #lower casing and removing extra spaces
          mylines.append(line2) 
  return mylines

#function for tokenizing text, removing stopword and stemming
def tokenizing(lines,stopword):
  corpus = " ".join(lines)
  corpus_tokenized = nltk.word_tokenize(corpus)
  corpus_tokenized = [word for word in corpus_tokenized if not word in stopword]   # removes stopwords
  corpus_tokenized = [ps.stem(word) for word in corpus_tokenized]  # keep the stem
  corpus_cleaned = " ".join(corpus_tokenized)
  corpus_cleaned =np.array(corpus_cleaned).reshape(-1)

  return corpus_cleaned

def count_length(file_name, start_page, end_page, out_name):
  text = mda_text_extractor(file_name,start_page, end_page)
  write_to_text(text,out_name)
  lines = prepare(out_name)
  corpus = " ".join(lines)
  corpus_tokenized = nltk.word_tokenize(corpus)
  n_words = len(corpus_tokenized)
  return n_words  
  


#function for applying tfidf vectorization
def tfidf_vsm(corpus_cleaned):
  vectorizer = TfidfVectorizer()
  x =  vectorizer.fit_transform(corpus_cleaned).toarray()
  x_df = pd.DataFrame(x)

  return x, x_df

#function for converting text into VSM model
def convert_to_vsm(file_name, start_page, end_page, out_name):
  text = mda_text_extractor(file_name,start_page, end_page)
  write_to_text(text,out_name)
  lines = prepare(out_name)
  corpus = " ".join(lines)
  corpus_tokenized = nltk.word_tokenize(corpus)
  n_words = len(corpus_tokenized)

  corpus_cleaned = tokenizing(lines,stopword)
  x , x_df = tfidf_vsm(corpus_cleaned)

  return x, x_df, n_words

#functions to calculate the final scores
def truncateData(data):
    truncLength = min([len(sym) for sym in data])
    return [x[len(x) - truncLength:] for x in data]

def calculate_score(x,y):
  lists = [x[0],y[0]]
  tr_list = truncateData(lists)
  score = sum([i*j for i,j in zip(x[0],y[0])])
  return score

def calculate_modification_score(doc1, start_page1, end_page1, out_name1,doc2, start_page2, end_page2, out_name2):
  x, x_df, n_words = convert_to_vsm(doc1, start_page1, end_page1, out_name1)
  y, y_df, n_words1 = convert_to_vsm(doc2, start_page2, end_page2, out_name2)

  ss = calculate_score(x,y)
  return  ss,n_words, n_words1


In [ ]:

dir1 = "/content/MD&A_Directory/MD&A Directory/"
all_files = os.listdir(dir1)



In [ ]:
len(all_files)

2

In [ ]:
key_names0 = ["".join(i.split(",")[0].split("_")[:-1]) for i in all_files] 
tmp = [x for x in key_names0 if str(x) != 'nan']
print(len(tmp))

2


In [ ]:
key_names1 = list(set(key_names0))

In [ ]:
len(key_names1)

1

In [ ]:
#putting the pdf files into seperate folders
for b in key_names1:
  try:
    pattern = b 
    matching_files = [f for f in all_files if pattern in "".join(f.split(",")[0].split("_")[:-1])]
    if len(matching_files) ==1:
      continue
    
    destinn = "/content/MD&A_Directory/MD&A Directory/"+ pattern + "/"
    os.makedirs(destinn, exist_ok= True)
    
    for fl in matching_files:
      os.rename("/content/MD&A_Directory/MD&A Directory/"+  fl , destinn + fl)
  except:
    print(b)
    
  

In [ ]:
import math

In [ ]:
import json

In [ ]:
pagesn = json.load(open('/content/drive/MyDrive/page_numbers_1.json')) #loads in the file containing the page numbers that the texts needs to be extracted from

In [ ]:
pagesn['Gran_Tierra_annual_report_Dec 31, 2015_Item 7_ONLY_p.58-82.pdf'] = [58,82]
pagesn['Gran_Tierra_annual_report_Dec 31, 2016_Item 7_ONLY_p.56-82.pdf'] = [56,82]
pagesn['Gran_Tierra_annual_report_Dec 31, 2017_Item 7_ONLY_p.34-57.pdf'] = [34,57]
pagesn['Power_Financial_Corporation_of_Canada_MD&A_Dec 31, 2015_p.4-44.pdf'] = [4,44]
pagesn['Power_Financial_Corporation_of_Canada_MD&A_Dec 31, 2016_p.4-47.pdf'] = [4,47]
pagesn['Power_Corporation_of_Canada_MD&A_Dec 31, 2015_p.4-50.pdf'] = [4,50]
pagesn['Power_Corporation_of_Canada_MD&A_Dec 31, 2016_p.4-52.pdf'] = [4,52]
pagesn['Power_Corporation_of_Canada_MD&A_Dec 31, 2017_p.4-62.pdf'] = [4,62]
pagesn['Klondex_Mines_Ltd_MD&A_Dec 31, 2016_Item_7_p.33-53.pdf'] = [33,53]
pagesn['Klondex_Mines_Ltd_MD&A_Dec 31, 2017_Item_7_p.34-58.pdf'] = [34,58]
pagesn['Cascade_MD&A_Dec 31, 2016_p.3-end.pdf'] = [3,1000]
pagesn['Cascade_MD&A_Dec 31, 2017_p.3-end.pdf'] = [3,1000]
pagesn['Primero_Mining_Dec 31, 2016_p20 -73 .pdf'] = [20,73]


In [ ]:
scores = {}

In [ ]:
#loop that iterates in the folders and calculate the score between two years
doc_length1 ={}
doc_length2 ={}
scores = {}
error = []

for fn in key_names1:
  try:
    
    dir_file ="/content/MD&A_Directory/MD&A Directory/" + fn + "/"     # Replace with path to your directory: absolute or relative
    pdf_names = os.listdir(dir_file)
    pdf_names = [g for g in pdf_names if g!='.ipynb_checkpoints']
    helper_dict = {}
    years = [2016,2017]
    for y, pds in enumerate(pdf_names):
      if y<2:
        name = os.path.splitext(pds)[0]
        ind = years[y]
        helper_dict[int(ind)] = pds
      else:
        continue
    for year in years[:-1]:
      try:
        ic = " ".join(name.split("_")[:3])
        id = "_".join([ic,str(year),str(year+1)])
        doc1 = dir_file +  helper_dict[year]
        try:
          start_page1 = pagesn[helper_dict[year]][0]
        except:
          start_page1 = 0
        try:
          end_page1 = pagesn[helper_dict[year]][1]
        except:
          end_page1 = 1000
        doc2 = dir_file + helper_dict[year+1]
        try:
          start_page2 = pagesn[helper_dict[year+1]][0]
        except:
          start_page2 = 0
        try:
          end_page2 = pagesn[helper_dict[year+1]][1]
        except:
          end_page2 =1000   
        out_name1 = helper_dict[year] + '.txt'
        out_name2 = helper_dict[year+1] + '.txt'
        id = "_".join([fn,str(year),str(year+1)])

        score,n1,n2=calculate_modification_score(doc1, start_page1, end_page1,out_name1,doc2, start_page2 , end_page2 , out_name2)
        print(id)
        print(1-score)
        scores[id]  = 1-score
        #doc_length1[id] =n1
        doc_length2[id] =n2
      except Exception as e:
        error.append(id)
        print(e)
        continue 
  except Exception as e:
    print(e)
    


PrimeroMiningMD&A_2016_2017
0.8372004960467611


'\nscores = {x:y for x,y in scores.items()}\nscores = {x:y for x,y in scores.items() if y!=0}\ndf = pd.DataFrame.from_dict(scores, orient=\'index\', columns = [\'Score\'])\n#df1 = pd.DataFrame.from_dict(doc_length1, orient=\'index\', columns = [\'n_words\'])\ndf2 = pd.DataFrame.from_dict(doc_length2, orient=\'index\', columns = [\'n_words\'])\ndf.rename_axis("Name", axis = \'columns\',inplace = True)\n\ndf.to_csv(\'/content/drive/MyDrive/MDA_SCORE_2018_2019_ext.csv\')\n#df1.to_csv(\'/content/drive/MyDrive/doc_length1.csv\')\ndf2.to_csv(\'/content/drive/MyDrive/doc_length_2018_2019_ext.csv\')\n'

In [ ]:

df0 = pd.read_csv('/content/drive/MyDrive/MDA_SCORE_2015_2016_ext.csv')
df0 = df0.rename(columns={'Unnamed: 0':'Company_Names','Score':'Raw_Score'})
df0 = df0[['Company_Names','Raw_Score']].reset_index(drop=True)
df0l = pd.read_csv('/content/drive/MyDrive/doc_length_2015_2016_ext.csv')
df0l = df0l.rename(columns={'Unnamed: 0':'Company_Names'})
df0l = df0l[['Company_Names','n_words']].reset_index(drop=True)
df0 = df0.merge(df0l,on='Company_Names')

df1 = pd.read_csv('/content/drive/MyDrive/MDA_SCORE_2016_2017_ext.csv')
df1 = df1.rename(columns={'Unnamed: 0':'Company_Names','Score':'Raw_Score'})
df1 = df1[['Company_Names','Raw_Score']].reset_index(drop=True)
df1l = pd.read_csv('/content/drive/MyDrive/doc_length_2016_2017_ext.csv')
df1l = df1l.rename(columns={'Unnamed: 0':'Company_Names'})
df1l = df1l[['Company_Names','n_words']].reset_index(drop=True)
df1 = df1.merge(df1l,on='Company_Names')

df2 = pd.read_csv('/content/drive/MyDrive/MDA_SCORE_2017_2018_ext.csv')
df2 = df2.rename(columns={'Unnamed: 0':'Company_Names','Score':'Raw_Score'})
df2 = df2[['Company_Names','Raw_Score']].reset_index(drop=True)
df2l = pd.read_csv('/content/drive/MyDrive/doc_length_2017_2018_ext.csv')
df2l = df2l.rename(columns={'Unnamed: 0':'Company_Names'})
df2l = df2l[['Company_Names','n_words']].reset_index(drop=True)
df2 = df2.merge(df2l,on='Company_Names')

df3 = pd.read_csv('/content/drive/MyDrive/MDA_SCORE_2018_2019_ext.csv')
df3 = df3.rename(columns={'Unnamed: 0':'Company_Names','Score':'Raw_Score'})
df3 = df3[['Company_Names','Raw_Score']].reset_index(drop=True)
df3l = pd.read_csv('/content/drive/MyDrive/doc_length_2018_2019_ext.csv')
df3l = df3l.rename(columns={'Unnamed: 0':'Company_Names'})
df3l = df3l[['Company_Names','n_words']].reset_index(drop=True)
df3 = df3.merge(df3l,on='Company_Names')
df_final = pd.concat([df0,df1,df2,df3],axis=0).reset_index(drop=True)

In [ ]:
df_final

,Company_Names,Raw_Score,n_words
0,AltagasMD&A_2015_2016,0.841735,33219
1,CrewEnergyMD&A_2016_2017,0.746125,10194
2,AltagasMD&A_2016_2017,0.760981,32144
3,CrewEnergyMD&A_2017_2018,0.717666,10194
4,AltagasMD&A_2017_2018,0.814733,32144
5,AltagasMD&A_2018_2019,0.834793,30326


In [ ]:
df_final.to_csv('/content/drive/MyDrive/MDA_SCORE_Final_ext.csv')

In [ ]:
df = pd.read_csv('/content/MDA_SCORE_FINAL2_UPDATED (CL).csv')

#reimporting the final file to calculate fitted score (the last update was dated on July 30, 2021). Second udpate Sept 4.

In [ ]:
df1 = df[['Company_Names','Raw_Score','n_words']].reset_index(drop=True)

In [ ]:
df.head()

,Match Key,index,Company_Names,Raw_Score,n_words,fitted_score,MD&A_SCORE,Unnamed: 7,Unnamed: 8
0,ADVANTAGE OIL & GAS LTD2016,259.0,AdvantageOilMD&A_2015_2016,0.782025,7595,0.78,0.002025,NaN,NaN
1,ADVANTAGE OIL & GAS LTD2017,537.0,AdvantageOilMD&A_2016_2017,0.740817,9122,0.78,-0.039183,Pass,NaN
2,ADVANTAGE OIL & GAS LTD2018,813.0,AdvantageOilMD&A_2017_2018,0.791845,9122,0.78,0.011845,Pass,NaN
3,ADVANTAGE OIL & GAS LTD2019,1093.0,AdvantageOilMD&A_2018_2019,0.782740,9846,0.78,0.002740,Pass,NaN
4,ADVANZ PHARMA CORP LTD2016,36.0,AdvanzPharmaMD&A_2015_2016,0.775292,19291,0.81,-0.034708,Pass,NaN


In [ ]:
from scipy.interpolate import approximate_taylor_polynomial
import numpy as np
from sklearn.linear_model import LinearRegression # sklearn linear model
from sklearn.metrics import mean_squared_error
#df = pd.read_csv('score_data.csv')
Y = df1['Raw_Score'].values
X = df1['n_words'].values
X_test = X
y = Y

In [ ]:

temp = X.copy()
unit = X_test.copy()
a = temp
b = unit

             # Iteration starts from the power of 2
for j in range(2, 6):

                 # Calculate the new column
      new_a = a ** j
      new_b = b ** j

                 # Stack new column
      temp = pd.concat((pd.DataFrame(temp), pd.DataFrame(new_a)), axis=1)
      unit = pd.concat((pd.DataFrame(unit), pd.DataFrame(new_b)), axis=1)

      temp = np.array(temp)
      unit = np.array(unit)

     # Import linear model
lr = LinearRegression()
lr.fit(temp, y)

     # Return the predicted value of the test set
fitted_score=lr.predict(unit)
fitted_score = fitted_score.round(2)
mda_score = Y - fitted_score

df1['fitted_score'] = fitted_score
df1['MD&A_SCORE'] = mda_score
#df.to_csv('/final_score_data.csv')

In [ ]:
df1.to_csv('/content/drive/MyDrive/MDA_SCORE_Final2.csv')

In [ ]:
# The coefficients
lr.coef_

array([ 2.53562491e-06, -7.93089448e-12,  6.56092870e-18,  2.12717088e-22,
        2.28629813e-23])

In [ ]:
lr.intercept_

0.761140207629408

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install linearmodels

     |████████████████████████████████| 1.5 MB 3.8 MB/s 
     |████████████████████████████████| 9.5 MB 38.9 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Jul  3 20:26:19 2021

@author: Amin.Masoudi

Updated August 13, 2021
"""

import pandas as pd

import statsmodels.formula.api as sm
#import statsmodels.api as sm1

import numpy as np

df = pd.read_excel('/Condensed Master Data Aug 7, 2021 (For Amin).xlsx')



dfd = df[['Revision','cusip','fyear','Car0','Car1','Car2','Car3','Scar0','Scar1','Scar2','Scar3','Car_e0','Car_e1','Car_e2','Car_e3','Scar_e0', 'Scar_e1', 'Scar_e2','Scar_e3','Other information MD&A Recoded','Score','Raw Score','Size','Filelate','NewItems (excluding 0)','POST']]
#CL: I added all of the relevant data columns
dfd.rename(columns={'Other information MD&A Recoded':'Auditor_OI','NewItems (excluding 0)':'NewItems'},inplace=True)
dfd['year'] = pd.to_datetime(dfd['fyear'],format='%Y')


dfd['AuScore'] = dfd['Auditor_OI'] * dfd['Score']

#Investors:
#Without CARea for Car0
#Score
cross_section_ols = sm.ols(formula='Car0 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore',
                          data=dfd).fit()

dfdd = dfd.dropna()
 
cross_section_mixedlm = sm.mixedlm(formula='Car0 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore',data=dfdd,groups=dfdd['cusip']).fit()

print('\nFirst model result\n')   
print('robust')                      
print(cross_section_ols.get_robustcov_results(cov_type='HC1').summary())
print('\nnon-robust\n')
print(cross_section_ols.summary())
print('\nmixed LM\n')
print(cross_section_mixedlm.summary())





/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a


First model result

robust
                            OLS Regression Results                            
Dep. Variable:                   Car0   R-squared:                       0.097
Model:                            OLS   Adj. R-squared:                  0.089
Method:                 Least Squares   F-statistic:                     92.20
Date:                Tue, 17 Aug 2021   Prob (F-statistic):          6.80e-139
Time:                        18:40:01   Log-Likelihood:                 1515.1
No. Observations:                1141   AIC:                            -3010.
Df Residuals:                    1131   BIC:                            -2960.
Df Model:                           9                                         
Covariance Type:                  HC1                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Jul  3 20:26:19 2021

@author: Amin.Masoudi

Updated August 13, 2021
"""

import pandas as pd

import statsmodels.formula.api as sm
#import statsmodels.api as sm1

import numpy as np

#df = pd.read_excel('/content/Condensed Master Data Aug 7, 2021 (For Amin).xlsx')
df = pd.read_excel('/Condensed Master Data Aug 7, 2021 (For Amin).xlsx')


dfd = df[['Revision','cusip','fyear','Car0','Car1','Car2','Car3','Scar0','Scar1','Scar2','Scar3','Car_e0','Car_e1','Car_e2','Car_e3','Scar_e0', 'Scar_e1', 'Scar_e2','Scar_e3','Other information MD&A Recoded','Score','Raw Score','Size','Filelate','NewItems (excluding 0)','POST']]
#CL: I added all of the relevant data columns
dfd.rename(columns={'Other information MD&A Recoded':'Auditor_OI','NewItems (excluding 0)':'NewItems'},inplace=True)
dfd['year'] = pd.to_datetime(dfd['fyear'],format='%Y')


dfd['AuScore'] = dfd['Auditor_OI'] * dfd['Score']

#Investors:
#Without CARea for Car0
#Score
cross_section_ols = sm.ols(formula='Car0 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore',
                          data=dfd).fit()

dfdd = dfd.dropna()
 
cross_section_mixedlm = sm.mixedlm(formula='Car0 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore',data=dfdd,groups=dfdd['cusip']).fit()

print('\nFirst model result\n')   
print('robust')                      
print(cross_section_ols.get_robustcov_results(cov_type='HC1').summary())
print('\nnon-robust\n')
print(cross_section_ols.summary())
print('\nmixed LM\n')
print(cross_section_mixedlm.summary())





/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to b


First model result

robust
                            OLS Regression Results                            
Dep. Variable:                   Car0   R-squared:                       0.097
Model:                            OLS   Adj. R-squared:                  0.089
Method:                 Least Squares   F-statistic:                     92.20
Date:                Tue, 17 Aug 2021   Prob (F-statistic):          6.80e-139
Time:                        18:26:20   Log-Likelihood:                 1515.1
No. Observations:                1141   AIC:                            -3010.
Df Residuals:                    1131   BIC:                            -2960.
Df Model:                           9                                         
Covariance Type:                  HC1                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

In [ ]:
#Investors:
#Without CARea for Car1
#Score

cross_section_ols = sm.ols(formula='Car1 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore',
                          data=dfd).fit()

dfdd = dfd.dropna()
 
cross_section_mixedlm = sm.mixedlm(formula='Car1 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore',data=dfdd,groups=dfdd['cusip']).fit()
print('\nFirst model result\n')   
print('robust')                      
print(cross_section_ols.get_robustcov_results(cov_type='HC1').summary())
print('\nnon-robust\n')
print(cross_section_ols.summary())
print('\nmixed LM\n')
print(cross_section_mixedlm.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 11, but rank is 10
  'rank is %d' % (J, J_), ValueWarning)



First model result

robust
                            OLS Regression Results                            
Dep. Variable:                   Car1   R-squared:                       0.128
Model:                            OLS   Adj. R-squared:                  0.121
Method:                 Least Squares   F-statistic:                     100.3
Date:                Fri, 13 Aug 2021   Prob (F-statistic):          2.60e-148
Time:                        20:43:34   Log-Likelihood:                 1591.7
No. Observations:                1141   AIC:                            -3163.
Df Residuals:                    1131   BIC:                            -3113.
Df Model:                           9                                         
Covariance Type:                  HC1                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

In [ ]:
#Investors:
#Without CARea for Car2
#Score

cross_section_ols = sm.ols(formula='Car2 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore',
                          data=dfd).fit()

dfdd = dfd.dropna()
 
cross_section_mixedlm = sm.mixedlm(formula='Car2 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore',data=dfdd,groups=dfdd['cusip']).fit()
print('\nFirst model result\n')   
print('robust')                      
print(cross_section_ols.get_robustcov_results(cov_type='HC1').summary())
print('\nnon-robust\n')
print(cross_section_ols.summary())
print('\nmixed LM\n')
print(cross_section_mixedlm.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 11, but rank is 10
  'rank is %d' % (J, J_), ValueWarning)



First model result

robust
                            OLS Regression Results                            
Dep. Variable:                   Car2   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.089
Method:                 Least Squares   F-statistic:                     93.08
Date:                Fri, 13 Aug 2021   Prob (F-statistic):          6.15e-140
Time:                        20:44:18   Log-Likelihood:                 1514.8
No. Observations:                1141   AIC:                            -3010.
Df Residuals:                    1131   BIC:                            -2959.
Df Model:                           9                                         
Covariance Type:                  HC1                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

In [ ]:
#Investors:
#Without CARea for Car3
#Score
cross_section_ols = sm.ols(formula='Car3 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore',
                          data=dfd).fit()

dfdd = dfd.dropna()
 
cross_section_mixedlm = sm.mixedlm(formula='Car3 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore',data=dfdd,groups=dfdd['cusip']).fit()
print('\nFirst model result\n')   
print('robust')                      
print(cross_section_ols.get_robustcov_results(cov_type='HC1').summary())
print('\nnon-robust\n')
print(cross_section_ols.summary())
print('\nmixed LM\n')
print(cross_section_mixedlm.summary())


First model result

robust
                            OLS Regression Results                            
Dep. Variable:                   Car3   R-squared:                       0.132
Model:                            OLS   Adj. R-squared:                  0.126
Method:                 Least Squares   F-statistic:                     101.1
Date:                Fri, 13 Aug 2021   Prob (F-statistic):          2.97e-149
Time:                        20:45:51   Log-Likelihood:                 1591.5
No. Observations:                1141   AIC:                            -3163.
Df Residuals:                    1131   BIC:                            -3113.
Df Model:                           9                                         
Covariance Type:                  HC1                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 11, but rank is 10
  'rank is %d' % (J, J_), ValueWarning)


In [ ]:
#Investors:
#Without CARea for Scar0
#Score

cross_section_ols = sm.ols(formula='Scar0 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore',
                          data=dfd).fit()

dfdd = dfd.dropna()
 
cross_section_mixedlm = sm.mixedlm(formula='Scar0 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore',data=dfdd,groups=dfdd['cusip']).fit()
print('\nFirst model result\n')   
print('robust')                      
print(cross_section_ols.get_robustcov_results(cov_type='HC1').summary())
print('\nnon-robust\n')
print(cross_section_ols.summary())
print('\nmixed LM\n')
print(cross_section_mixedlm.summary())


First model result

robust
                            OLS Regression Results                            
Dep. Variable:                  Scar0   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     113.1
Date:                Fri, 13 Aug 2021   Prob (F-statistic):          1.64e-162
Time:                        20:46:32   Log-Likelihood:                -2208.2
No. Observations:                1141   AIC:                             4436.
Df Residuals:                    1131   BIC:                             4487.
Df Model:                           9                                         
Covariance Type:                  HC1                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 11, but rank is 10
  'rank is %d' % (J, J_), ValueWarning)


In [ ]:
#Investors:
#Without CARea for Scar1
#Score

cross_section_ols = sm.ols(formula='Scar1 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore',
                          data=dfd).fit()

dfdd = dfd.dropna()
 
cross_section_mixedlm = sm.mixedlm(formula='Scar1 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore',data=dfdd,groups=dfdd['cusip']).fit()
print('\nFirst model result\n')   
print('robust')                      
print(cross_section_ols.get_robustcov_results(cov_type='HC1').summary())
print('\nnon-robust\n')
print(cross_section_ols.summary())
print('\nmixed LM\n')
print(cross_section_mixedlm.summary())


First model result

robust
                            OLS Regression Results                            
Dep. Variable:                  Scar1   R-squared:                       0.054
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     118.4
Date:                Fri, 13 Aug 2021   Prob (F-statistic):          5.03e-168
Time:                        20:48:51   Log-Likelihood:                -2212.0
No. Observations:                1138   AIC:                             4444.
Df Residuals:                    1128   BIC:                             4494.
Df Model:                           9                                         
Covariance Type:                  HC1                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 11, but rank is 10
  'rank is %d' % (J, J_), ValueWarning)


In [ ]:
#Investors:
#Without CARea for Scar2
#Score

cross_section_ols = sm.ols(formula='Scar2 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore',
                          data=dfd).fit()

dfdd = dfd.dropna()
 
cross_section_mixedlm = sm.mixedlm(formula='Scar2 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore',data=dfdd,groups=dfdd['cusip']).fit()
print('\nFirst model result\n')   
print('robust')                      
print(cross_section_ols.get_robustcov_results(cov_type='HC1').summary())
print('\nnon-robust\n')
print(cross_section_ols.summary())
print('\nmixed LM\n')
print(cross_section_mixedlm.summary())


First model result

robust
                            OLS Regression Results                            
Dep. Variable:                  Scar2   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     112.4
Date:                Fri, 13 Aug 2021   Prob (F-statistic):          9.94e-162
Time:                        20:49:36   Log-Likelihood:                -2217.9
No. Observations:                1141   AIC:                             4456.
Df Residuals:                    1131   BIC:                             4506.
Df Model:                           9                                         
Covariance Type:                  HC1                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 11, but rank is 10
  'rank is %d' % (J, J_), ValueWarning)


In [ ]:
#Investors:
#Without CARea for Scar3
#Score

cross_section_ols = sm.ols(formula='Scar3 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore',
                          data=dfd).fit()

dfdd = dfd.dropna()
 
cross_section_mixedlm = sm.mixedlm(formula='Scar3 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore',data=dfdd,groups=dfdd['cusip']).fit()
print('\nFirst model result\n')   
print('robust')                      
print(cross_section_ols.get_robustcov_results(cov_type='HC1').summary())
print('\nnon-robust\n')
print(cross_section_ols.summary())
print('\nmixed LM\n')
print(cross_section_mixedlm.summary())


First model result

robust
                            OLS Regression Results                            
Dep. Variable:                  Scar3   R-squared:                       0.054
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     117.9
Date:                Fri, 13 Aug 2021   Prob (F-statistic):          1.20e-167
Time:                        20:50:04   Log-Likelihood:                -2225.9
No. Observations:                1141   AIC:                             4472.
Df Residuals:                    1131   BIC:                             4522.
Df Model:                           9                                         
Covariance Type:                  HC1                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 11, but rank is 10
  'rank is %d' % (J, J_), ValueWarning)


In [ ]:
#Investors:
#With CARea Car0 - Car_e0
#Score

cross_section_ols = sm.ols(formula='Car0 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore + Car_e0',
                          data=dfd).fit()

dfdd = dfd.dropna()
 
cross_section_mixedlm = sm.mixedlm(formula='Car0 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore+Car_e0',data=dfdd,groups=dfdd['cusip']).fit()

print('\nFirst model result\n')   
print('robust')                      
print(cross_section_ols.get_robustcov_results(cov_type='HC1').summary())
print('\nnon-robust\n')
print(cross_section_ols.summary())
print('\nmixed LM\n')
print(cross_section_mixedlm.summary())


/usr/local/lib/python3.7/dist-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 12, but rank is 11
  'rank is %d' % (J, J_), ValueWarning)



First model result

robust
                            OLS Regression Results                            
Dep. Variable:                   Car0   R-squared:                       0.834
Model:                            OLS   Adj. R-squared:                  0.833
Method:                 Least Squares   F-statistic:                     825.1
Date:                Fri, 13 Aug 2021   Prob (F-statistic):               0.00
Time:                        20:53:42   Log-Likelihood:                 2482.6
No. Observations:                1141   AIC:                            -4943.
Df Residuals:                    1130   BIC:                            -4888.
Df Model:                          10                                         
Covariance Type:                  HC1                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

In [ ]:
#Investors:
#With CARea Car1 - Car_e1
#Score

cross_section_ols = sm.ols(formula='Car1 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore + Car_e1',
                          data=dfd).fit()

dfdd = dfd.dropna()
 
cross_section_mixedlm = sm.mixedlm(formula='Car1 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore+Car_e1',data=dfdd,groups=dfdd['cusip']).fit()

print('\nFirst model result\n')   
print('robust')                      
print(cross_section_ols.get_robustcov_results(cov_type='HC1').summary())
print('\nnon-robust\n')
print(cross_section_ols.summary())
print('\nmixed LM\n')
print(cross_section_mixedlm.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 12, but rank is 11
  'rank is %d' % (J, J_), ValueWarning)



First model result

robust
                            OLS Regression Results                            
Dep. Variable:                   Car1   R-squared:                       0.822
Model:                            OLS   Adj. R-squared:                  0.820
Method:                 Least Squares   F-statistic:                     813.4
Date:                Fri, 13 Aug 2021   Prob (F-statistic):               0.00
Time:                        20:55:11   Log-Likelihood:                 2497.1
No. Observations:                1141   AIC:                            -4972.
Df Residuals:                    1130   BIC:                            -4917.
Df Model:                          10                                         
Covariance Type:                  HC1                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

In [ ]:
#Investors:
#With CARea Car2 - Car_e2
#Score

cross_section_ols = sm.ols(formula='Car2 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore + Car_e2',
                          data=dfd).fit()

dfdd = dfd.dropna()
 
cross_section_mixedlm = sm.mixedlm(formula='Car2 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore+Car_e2',data=dfdd,groups=dfdd['cusip']).fit()

print('\nFirst model result\n')   
print('robust')                      
print(cross_section_ols.get_robustcov_results(cov_type='HC1').summary())
print('\nnon-robust\n')
print(cross_section_ols.summary())
print('\nmixed LM\n')
print(cross_section_mixedlm.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 12, but rank is 11
  'rank is %d' % (J, J_), ValueWarning)



First model result

robust
                            OLS Regression Results                            
Dep. Variable:                   Car2   R-squared:                       0.834
Model:                            OLS   Adj. R-squared:                  0.832
Method:                 Least Squares   F-statistic:                     839.2
Date:                Fri, 13 Aug 2021   Prob (F-statistic):               0.00
Time:                        20:55:48   Log-Likelihood:                 2480.5
No. Observations:                1141   AIC:                            -4939.
Df Residuals:                    1130   BIC:                            -4884.
Df Model:                          10                                         
Covariance Type:                  HC1                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

In [ ]:
#Investors:
#With CARea Car3 - Car_e3
#Score

cross_section_ols = sm.ols(formula='Car3 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore + Car_e3',
                          data=dfd).fit()

dfdd = dfd.dropna()
 
cross_section_mixedlm = sm.mixedlm(formula='Car3 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore+Car_e3',data=dfdd,groups=dfdd['cusip']).fit()

print('\nFirst model result\n')   
print('robust')                      
print(cross_section_ols.get_robustcov_results(cov_type='HC1').summary())
print('\nnon-robust\n')
print(cross_section_ols.summary())
print('\nmixed LM\n')
print(cross_section_mixedlm.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 12, but rank is 11
  'rank is %d' % (J, J_), ValueWarning)



First model result

robust
                            OLS Regression Results                            
Dep. Variable:                   Car3   R-squared:                       0.818
Model:                            OLS   Adj. R-squared:                  0.816
Method:                 Least Squares   F-statistic:                     774.4
Date:                Fri, 13 Aug 2021   Prob (F-statistic):               0.00
Time:                        20:57:01   Log-Likelihood:                 2481.7
No. Observations:                1141   AIC:                            -4941.
Df Residuals:                    1130   BIC:                            -4886.
Df Model:                          10                                         
Covariance Type:                  HC1                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

In [ ]:
#Investors:
#With CARea Scar0 - Scar_e0
#Score

cross_section_ols = sm.ols(formula='Scar0 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore + Scar_e0',
                          data=dfd).fit()

dfdd = dfd.dropna()
 
cross_section_mixedlm = sm.mixedlm(formula='Scar0 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore+Scar_e0',data=dfdd,groups=dfdd['cusip']).fit()

print('\nFirst model result\n')   
print('robust')                      
print(cross_section_ols.get_robustcov_results(cov_type='HC1').summary())
print('\nnon-robust\n')
print(cross_section_ols.summary())
print('\nmixed LM\n')
print(cross_section_mixedlm.summary())


First model result

robust
                            OLS Regression Results                            
Dep. Variable:                  Scar0   R-squared:                       0.807
Model:                            OLS   Adj. R-squared:                  0.805
Method:                 Least Squares   F-statistic:                     998.1
Date:                Fri, 13 Aug 2021   Prob (F-statistic):               0.00
Time:                        20:59:03   Log-Likelihood:                -1294.5
No. Observations:                1141   AIC:                             2611.
Df Residuals:                    1130   BIC:                             2666.
Df Model:                          10                                         
Covariance Type:                  HC1                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 12, but rank is 11
  'rank is %d' % (J, J_), ValueWarning)


In [ ]:
#Investors:
#With CARea Scar1 - Scar_e1
#Score

cross_section_ols = sm.ols(formula='Scar1 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore + Scar_e1',
                          data=dfd).fit()

dfdd = dfd.dropna()
 
cross_section_mixedlm = sm.mixedlm(formula='Scar1 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore+Scar_e1',data=dfdd,groups=dfdd['cusip']).fit()

print('\nFirst model result\n')   
print('robust')                      
print(cross_section_ols.get_robustcov_results(cov_type='HC1').summary())
print('\nnon-robust\n')
print(cross_section_ols.summary())
print('\nmixed LM\n')
print(cross_section_mixedlm.summary())


First model result

robust
                            OLS Regression Results                            
Dep. Variable:                  Scar1   R-squared:                       0.778
Model:                            OLS   Adj. R-squared:                  0.776
Method:                 Least Squares   F-statistic:                     743.3
Date:                Fri, 13 Aug 2021   Prob (F-statistic):               0.00
Time:                        21:00:08   Log-Likelihood:                -1387.2
No. Observations:                1138   AIC:                             2796.
Df Residuals:                    1127   BIC:                             2852.
Df Model:                          10                                         
Covariance Type:                  HC1                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 12, but rank is 11
  'rank is %d' % (J, J_), ValueWarning)


In [ ]:
#Investors:
#With CARea Scar2 - Scar_e2
#Score

cross_section_ols = sm.ols(formula='Scar2 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore + Scar_e2',
                          data=dfd).fit()

dfdd = dfd.dropna()
 
cross_section_mixedlm = sm.mixedlm(formula='Scar2 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore+Scar_e2',data=dfdd,groups=dfdd['cusip']).fit()

print('\nFirst model result\n')   
print('robust')                      
print(cross_section_ols.get_robustcov_results(cov_type='HC1').summary())
print('\nnon-robust\n')
print(cross_section_ols.summary())
print('\nmixed LM\n')
print(cross_section_mixedlm.summary())


First model result

robust
                            OLS Regression Results                            
Dep. Variable:                  Scar2   R-squared:                       0.805
Model:                            OLS   Adj. R-squared:                  0.803
Method:                 Least Squares   F-statistic:                     965.8
Date:                Fri, 13 Aug 2021   Prob (F-statistic):               0.00
Time:                        21:01:23   Log-Likelihood:                -1308.6
No. Observations:                1141   AIC:                             2639.
Df Residuals:                    1130   BIC:                             2695.
Df Model:                          10                                         
Covariance Type:                  HC1                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 12, but rank is 11
  'rank is %d' % (J, J_), ValueWarning)


In [ ]:
#Investors:
#With CARea Scar3 - Scar_e3
#Score

cross_section_ols = sm.ols(formula='Scar3 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore + Scar_e3',
                          data=dfd).fit()

dfdd = dfd.dropna()
 
cross_section_mixedlm = sm.mixedlm(formula='Scar3 ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore+Scar_e3',data=dfdd,groups=dfdd['cusip']).fit()

print('\nFirst model result\n')   
print('robust')                      
print(cross_section_ols.get_robustcov_results(cov_type='HC1').summary())
print('\nnon-robust\n')
print(cross_section_ols.summary())
print('\nmixed LM\n')
print(cross_section_mixedlm.summary())


First model result

robust
                            OLS Regression Results                            
Dep. Variable:                  Scar3   R-squared:                       0.773
Model:                            OLS   Adj. R-squared:                  0.771
Method:                 Least Squares   F-statistic:                     716.2
Date:                Fri, 13 Aug 2021   Prob (F-statistic):               0.00
Time:                        21:02:03   Log-Likelihood:                -1410.6
No. Observations:                1141   AIC:                             2843.
Df Residuals:                    1130   BIC:                             2899.
Df Model:                          10                                         
Covariance Type:                  HC1                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 12, but rank is 11
  'rank is %d' % (J, J_), ValueWarning)


In [ ]:
#Analysts:

#Without CARea
#Score

cross_section_ols = sm.ols(formula='Revision ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore',
                          data=dfd).fit()

dfdd = dfd.dropna()
 
cross_section_mixedlm = sm.mixedlm(formula='Revision ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore',data=dfdd,groups=dfdd['cusip']).fit()

print('\nFirst model result\n')   
print('robust')                      
print(cross_section_ols.get_robustcov_results(cov_type='HC1').summary())
print('\nnon-robust\n')
print(cross_section_ols.summary())
print('\nmixed LM\n')
print(cross_section_mixedlm.summary())


First model result

robust
                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     13.95
Date:                Fri, 13 Aug 2021   Prob (F-statistic):           5.89e-23
Time:                        21:04:05   Log-Likelihood:                 696.63
No. Observations:                 834   AIC:                            -1373.
Df Residuals:                     824   BIC:                            -1326.
Df Model:                           9                                         
Covariance Type:                  HC1                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 11, but rank is 10
  'rank is %d' % (J, J_), ValueWarning)


In [ ]:
#Analysts:

#With CARea - Car_e0
#Score

cross_section_ols = sm.ols(formula='Revision ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore + Car_e0',
                          data=dfd).fit()

dfdd = dfd.dropna()
 
cross_section_mixedlm = sm.mixedlm(formula='Revision ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore + Car_e0',data=dfdd,groups=dfdd['cusip']).fit()

print('\nFirst model result\n')   
print('robust')                      
print(cross_section_ols.get_robustcov_results(cov_type='HC1').summary())
print('\nnon-robust\n')
print(cross_section_ols.summary())
print('\nmixed LM\n')
print(cross_section_mixedlm.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 12, but rank is 11
  'rank is %d' % (J, J_), ValueWarning)



First model result

robust
                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     13.17
Date:                Fri, 13 Aug 2021   Prob (F-statistic):           2.46e-23
Time:                        21:06:25   Log-Likelihood:                 701.14
No. Observations:                 833   AIC:                            -1380.
Df Residuals:                     822   BIC:                            -1328.
Df Model:                          10                                         
Covariance Type:                  HC1                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

In [ ]:
#Analysts:

#With CARea - Car_e1
#Score

cross_section_ols = sm.ols(formula='Revision ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore + Car_e1',
                          data=dfd).fit()

dfdd = dfd.dropna()
 
cross_section_mixedlm = sm.mixedlm(formula='Revision ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore + Car_e1',data=dfdd,groups=dfdd['cusip']).fit()

print('\nFirst model result\n')   
print('robust')                      
print(cross_section_ols.get_robustcov_results(cov_type='HC1').summary())
print('\nnon-robust\n')
print(cross_section_ols.summary())
print('\nmixed LM\n')
print(cross_section_mixedlm.summary())


First model result

robust
                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     12.76
Date:                Fri, 13 Aug 2021   Prob (F-statistic):           1.55e-22
Time:                        21:06:57   Log-Likelihood:                 699.50
No. Observations:                 833   AIC:                            -1377.
Df Residuals:                     822   BIC:                            -1325.
Df Model:                          10                                         
Covariance Type:                  HC1                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 12, but rank is 11
  'rank is %d' % (J, J_), ValueWarning)


In [ ]:
#Analysts:

#With CARea - Car_e2
#Score

cross_section_ols = sm.ols(formula='Revision ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore + Car_e2',
                          data=dfd).fit()

dfdd = dfd.dropna()
 
cross_section_mixedlm = sm.mixedlm(formula='Revision ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore + Car_e2',data=dfdd,groups=dfdd['cusip']).fit()

print('\nFirst model result\n')   
print('robust')                      
print(cross_section_ols.get_robustcov_results(cov_type='HC1').summary())
print('\nnon-robust\n')
print(cross_section_ols.summary())
print('\nmixed LM\n')
print(cross_section_mixedlm.summary())


First model result

robust
                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     13.20
Date:                Fri, 13 Aug 2021   Prob (F-statistic):           2.24e-23
Time:                        21:07:22   Log-Likelihood:                 701.28
No. Observations:                 833   AIC:                            -1381.
Df Residuals:                     822   BIC:                            -1329.
Df Model:                          10                                         
Covariance Type:                  HC1                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 12, but rank is 11
  'rank is %d' % (J, J_), ValueWarning)


In [ ]:
#Analysts:

#With CARea - Car_e3
#Score

cross_section_ols = sm.ols(formula='Revision ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore + Car_e3',
                          data=dfd).fit()

dfdd = dfd.dropna()
 
cross_section_mixedlm = sm.mixedlm(formula='Revision ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore + Car_e3',data=dfdd,groups=dfdd['cusip']).fit()

print('\nFirst model result\n')   
print('robust')                      
print(cross_section_ols.get_robustcov_results(cov_type='HC1').summary())
print('\nnon-robust\n')
print(cross_section_ols.summary())
print('\nmixed LM\n')
print(cross_section_mixedlm.summary())


First model result

robust
                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     12.75
Date:                Fri, 13 Aug 2021   Prob (F-statistic):           1.56e-22
Time:                        21:07:50   Log-Likelihood:                 699.50
No. Observations:                 833   AIC:                            -1377.
Df Residuals:                     822   BIC:                            -1325.
Df Model:                          10                                         
Covariance Type:                  HC1                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 12, but rank is 11
  'rank is %d' % (J, J_), ValueWarning)


In [ ]:
#Analysts:

#With CARea - Scar_e0
#Score

cross_section_ols = sm.ols(formula='Revision ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore + Scar_e0',
                          data=dfd).fit()

dfdd = dfd.dropna()
 
cross_section_mixedlm = sm.mixedlm(formula='Revision ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore + Scar_e0',data=dfdd,groups=dfdd['cusip']).fit()

print('\nFirst model result\n')   
print('robust')                      
print(cross_section_ols.get_robustcov_results(cov_type='HC1').summary())
print('\nnon-robust\n')
print(cross_section_ols.summary())
print('\nmixed LM\n')
print(cross_section_mixedlm.summary())


First model result

robust
                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     13.48
Date:                Fri, 13 Aug 2021   Prob (F-statistic):           6.32e-24
Time:                        21:08:35   Log-Likelihood:                 695.45
No. Observations:                 833   AIC:                            -1369.
Df Residuals:                     822   BIC:                            -1317.
Df Model:                          10                                         
Covariance Type:                  HC1                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 12, but rank is 11
  'rank is %d' % (J, J_), ValueWarning)


In [ ]:
#Analysts:

#With CARea - Scar_e1
#Score

cross_section_ols = sm.ols(formula='Revision ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore + Scar_e1',
                          data=dfd).fit()

dfdd = dfd.dropna()
 
cross_section_mixedlm = sm.mixedlm(formula='Revision ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore + Scar_e1',data=dfdd,groups=dfdd['cusip']).fit()

print('\nFirst model result\n')   
print('robust')                      
print(cross_section_ols.get_robustcov_results(cov_type='HC1').summary())
print('\nnon-robust\n')
print(cross_section_ols.summary())
print('\nmixed LM\n')
print(cross_section_mixedlm.summary())


First model result

robust
                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     14.08
Date:                Fri, 13 Aug 2021   Prob (F-statistic):           4.85e-25
Time:                        21:09:05   Log-Likelihood:                 692.04
No. Observations:                 830   AIC:                            -1362.
Df Residuals:                     819   BIC:                            -1310.
Df Model:                          10                                         
Covariance Type:                  HC1                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 12, but rank is 11
  'rank is %d' % (J, J_), ValueWarning)


In [ ]:
#Analysts:

#With CARea - Scar_e2
#Score

cross_section_ols = sm.ols(formula='Revision ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore + Scar_e2',
                          data=dfd).fit()

dfdd = dfd.dropna()
 
cross_section_mixedlm = sm.mixedlm(formula='Revision ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore + Scar_e2',data=dfdd,groups=dfdd['cusip']).fit()

print('\nFirst model result\n')   
print('robust')                      
print(cross_section_ols.get_robustcov_results(cov_type='HC1').summary())
print('\nnon-robust\n')
print(cross_section_ols.summary())
print('\nmixed LM\n')
print(cross_section_mixedlm.summary())


First model result

robust
                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     13.49
Date:                Fri, 13 Aug 2021   Prob (F-statistic):           6.24e-24
Time:                        21:09:39   Log-Likelihood:                 695.44
No. Observations:                 833   AIC:                            -1369.
Df Residuals:                     822   BIC:                            -1317.
Df Model:                          10                                         
Covariance Type:                  HC1                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 12, but rank is 11
  'rank is %d' % (J, J_), ValueWarning)


In [ ]:
#Analysts:

#With CARea - Scar_e3
#Score

cross_section_ols = sm.ols(formula='Revision ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore + Scar_e3',
                          data=dfd).fit()

dfdd = dfd.dropna()
 
cross_section_mixedlm = sm.mixedlm(formula='Revision ~ Score+ Size+ C(year)+ Filelate+ NewItems+ Auditor_OI+ AuScore + Scar_e3',data=dfdd,groups=dfdd['cusip']).fit()

print('\nFirst model result\n')   
print('robust')                      
print(cross_section_ols.get_robustcov_results(cov_type='HC1').summary())
print('\nnon-robust\n')
print(cross_section_ols.summary())
print('\nmixed LM\n')
print(cross_section_mixedlm.summary())


First model result

robust
                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     14.22
Date:                Fri, 13 Aug 2021   Prob (F-statistic):           2.58e-25
Time:                        21:10:17   Log-Likelihood:                 695.93
No. Observations:                 833   AIC:                            -1370.
Df Residuals:                     822   BIC:                            -1318.
Df Model:                          10                                         
Covariance Type:                  HC1                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 12, but rank is 11
  'rank is %d' % (J, J_), ValueWarning)


In [ ]:
chek1 = dfd.isnull()
#chek1[chek1=='True']
chek1

,Revision,cusip,fyear,Car0,Car1,Car2,Car3,Scar0,Scar1,Scar2,Scar3,Car_e0,Car_e1,Car_e2,Car_e3,Scar_e0,Scar_e1,Scar_e2,Scar_e3,Auditor_OI,Score,Raw Score,Size,Filelate,NewItems,POST,year,AuScore
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,True,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1359,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1360,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1361,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1362,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
#Time-series fixed effects(second model)

import pandas as pd

import statsmodels.formula.api as sm
#import statsmodels.api as sm1

import numpy as np

df = pd.read_excel('/content/Treatment Group August 7, 2021.xlsx') 

dfd = df[['Revision','cusip','fyear','Car0','Car1','Car2','Car3','Scar0','Scar1','Scar2','Scar3','Car_e0','Car_e1','Car_e2','Car_e3','Scar_e0', 'Scar_e1', 'Scar_e2','Scar_e3','Other information MD&A Recoded','Score','Raw Score','Size','Filelate','NewItems (excluding 0)','POST']]
#CL: I added all of the relevant data columns
dfd.rename(columns={'Other information MD&A Recoded':'Auditor_OI','NewItems (excluding 0)':'NewItems'},inplace=True)
dfd['year'] = pd.to_datetime(dfd['fyear'],format='%Y')


dfd['POScore'] = dfd['POST'] * dfd['Score']

#Without CARea
#Score

for i in ['Car0','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Car0 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ NewItems+ POST + POScore',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())



/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st


Second model result

                            OLS Regression Results                            
Dep. Variable:                   Car0   R-squared:                       0.346
Model:                            OLS   Adj. R-squared:                  0.093
Method:                 Least Squares   F-statistic:                     1.369
Date:                Sun, 15 Aug 2021   Prob (F-statistic):            0.00295
Time:                        01:36:30   Log-Likelihood:                 1203.6
No. Observations:                 725   AIC:                            -2001.
Df Residuals:                     522   BIC:                            -1070.
Df Model:                         202                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

In [ ]:
#Investors
#Treatment Group
#Without CARea
#Score
#Car1

for i in ['Car1','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Car1 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ NewItems+ POST + POScore',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                   Car1   R-squared:                       0.358
Model:                            OLS   Adj. R-squared:                  0.109
Method:                 Least Squares   F-statistic:                     1.438
Date:                Fri, 13 Aug 2021   Prob (F-statistic):           0.000698
Time:                        21:15:32   Log-Likelihood:                 1233.9
No. Observations:                 725   AIC:                            -2062.
Df Residuals:                     522   BIC:                            -1131.
Df Model:                         202                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#Investors
#Treatment Group
#Without CARea
#Score
#Car2

for i in ['Car2','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Car2 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ NewItems+ POST + POScore',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())



Second model result

                            OLS Regression Results                            
Dep. Variable:                   Car2   R-squared:                       0.345
Model:                            OLS   Adj. R-squared:                  0.092
Method:                 Least Squares   F-statistic:                     1.363
Date:                Sun, 15 Aug 2021   Prob (F-statistic):            0.00330
Time:                        01:40:24   Log-Likelihood:                 1202.3
No. Observations:                 725   AIC:                            -1999.
Df Residuals:                     522   BIC:                            -1068.
Df Model:                         202                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investors
#Treatment Group
#Without CARea
#Score
#Car3

for i in ['Car3','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Car3 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ NewItems+ POST + POScore',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())



Second model result

                            OLS Regression Results                            
Dep. Variable:                   Car3   R-squared:                       0.359
Model:                            OLS   Adj. R-squared:                  0.111
Method:                 Least Squares   F-statistic:                     1.450
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           0.000549
Time:                        01:41:08   Log-Likelihood:                 1235.7
No. Observations:                 725   AIC:                            -2065.
Df Residuals:                     522   BIC:                            -1134.
Df Model:                         202                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investors
#Treatment Group
#Without CARea
#Score
#Scar0

for i in ['Scar0','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Scar0 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ NewItems+ POST + POScore',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())



Second model result

                            OLS Regression Results                            
Dep. Variable:                  Scar0   R-squared:                       0.308
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     1.151
Date:                Sun, 15 Aug 2021   Prob (F-statistic):              0.109
Time:                        01:44:06   Log-Likelihood:                -1282.1
No. Observations:                 725   AIC:                             2970.
Df Residuals:                     522   BIC:                             3901.
Df Model:                         202                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investors
#Treatment Group
#Without CARea
#Score
#Scar1

for i in ['Scar1','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Scar1 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ NewItems+ POST + POScore',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                  Scar1   R-squared:                       0.309
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     1.153
Date:                Sun, 15 Aug 2021   Prob (F-statistic):              0.106
Time:                        01:44:31   Log-Likelihood:                -1302.0
No. Observations:                 725   AIC:                             3010.
Df Residuals:                     522   BIC:                             3941.
Df Model:                         202                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investors
#Treatment Group
#Without CARea
#Score
#Scar2

for i in ['Scar2','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Scar2 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ NewItems+ POST + POScore',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                  Scar2   R-squared:                       0.311
Model:                            OLS   Adj. R-squared:                  0.044
Method:                 Least Squares   F-statistic:                     1.166
Date:                Sun, 15 Aug 2021   Prob (F-statistic):             0.0899
Time:                        01:45:06   Log-Likelihood:                -1287.4
No. Observations:                 725   AIC:                             2981.
Df Residuals:                     522   BIC:                             3912.
Df Model:                         202                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investors
#Treatment Group
#Without CARea
#Score
#Scar3

for i in ['Scar3','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Scar3 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ NewItems+ POST + POScore',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                  Scar3   R-squared:                       0.312
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     1.173
Date:                Sun, 15 Aug 2021   Prob (F-statistic):             0.0820
Time:                        01:45:43   Log-Likelihood:                -1305.0
No. Observations:                 725   AIC:                             3016.
Df Residuals:                     522   BIC:                             3947.
Df Model:                         202                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investors
#Treatment Group
#With CARea
#Score
#Car0 + Car_e0

for i in ['Car0','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Car_e0']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Car0 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ NewItems+ POST + POScore + Car_e0',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                   Car0   R-squared:                       0.926
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     31.88
Date:                Sun, 15 Aug 2021   Prob (F-statistic):          1.66e-206
Time:                        01:46:57   Log-Likelihood:                 1990.9
No. Observations:                 725   AIC:                            -3574.
Df Residuals:                     521   BIC:                            -2638.
Df Model:                         203                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investors
#Treatment Group
#With CARea
#Score
#Car1 + Car_e1

for i in ['Car1','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Car_e1']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Car1 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ NewItems+ POST + POScore + Car_e1',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                   Car1   R-squared:                       0.905
Model:                            OLS   Adj. R-squared:                  0.869
Method:                 Least Squares   F-statistic:                     24.57
Date:                Sun, 15 Aug 2021   Prob (F-statistic):          1.91e-180
Time:                        01:47:42   Log-Likelihood:                 1928.3
No. Observations:                 725   AIC:                            -3449.
Df Residuals:                     521   BIC:                            -2513.
Df Model:                         203                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investors
#Treatment Group
#With CARea
#Score
#Car2 + Car_e2

for i in ['Car2','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Car_e2']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Car2 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ NewItems+ POST + POScore + Car_e2',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                   Car2   R-squared:                       0.926
Model:                            OLS   Adj. R-squared:                  0.897
Method:                 Least Squares   F-statistic:                     31.98
Date:                Sun, 15 Aug 2021   Prob (F-statistic):          8.01e-207
Time:                        01:48:18   Log-Likelihood:                 1991.2
No. Observations:                 725   AIC:                            -3574.
Df Residuals:                     521   BIC:                            -2639.
Df Model:                         203                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investors
#Treatment Group
#With CARea
#Score
#Car3 + Car_e3

for i in ['Car3','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Car_e3']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Car3 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ NewItems+ POST + POScore + Car_e3',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                   Car3   R-squared:                       0.904
Model:                            OLS   Adj. R-squared:                  0.866
Method:                 Least Squares   F-statistic:                     24.07
Date:                Sun, 15 Aug 2021   Prob (F-statistic):          1.97e-178
Time:                        01:48:42   Log-Likelihood:                 1922.4
No. Observations:                 725   AIC:                            -3437.
Df Residuals:                     521   BIC:                            -2501.
Df Model:                         203                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investors
#Treatment Group
#With CARea
#Score
#Scar0 + Scar_e0

for i in ['Scar0','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Scar_e0']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Scar0 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ NewItems+ POST + POScore + Scar_e0',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                  Scar0   R-squared:                       0.925
Model:                            OLS   Adj. R-squared:                  0.895
Method:                 Least Squares   F-statistic:                     31.46
Date:                Sun, 15 Aug 2021   Prob (F-statistic):          3.74e-205
Time:                        01:49:22   Log-Likelihood:                -478.73
No. Observations:                 725   AIC:                             1365.
Df Residuals:                     521   BIC:                             2301.
Df Model:                         203                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investors
#Treatment Group
#With CARea
#Score
#Scar1 + Scar_e1

for i in ['Scar1','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Scar_e1']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Scar1 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ NewItems+ POST + POScore + Scar_e1',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                  Scar1   R-squared:                       0.905
Model:                            OLS   Adj. R-squared:                  0.867
Method:                 Least Squares   F-statistic:                     24.34
Date:                Sun, 15 Aug 2021   Prob (F-statistic):          1.50e-179
Time:                        01:51:57   Log-Likelihood:                -583.90
No. Observations:                 725   AIC:                             1576.
Df Residuals:                     521   BIC:                             2511.
Df Model:                         203                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investors
#Treatment Group
#With CARea
#Score
#Scar2 + Scar_e2

for i in ['Scar2','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Scar_e2']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Scar2 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ NewItems+ POST + POScore + Scar_e2',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                  Scar2   R-squared:                       0.925
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     31.62
Date:                Sun, 15 Aug 2021   Prob (F-statistic):          1.19e-205
Time:                        01:52:18   Log-Likelihood:                -483.78
No. Observations:                 725   AIC:                             1376.
Df Residuals:                     521   BIC:                             2311.
Df Model:                         203                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investors
#Treatment Group
#With CARea
#Score
#Scar3 + Scar_e3

for i in ['Scar3','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Scar_e3']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Scar3 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ NewItems+ POST + POScore + Scar_e3',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                  Scar3   R-squared:                       0.903
Model:                            OLS   Adj. R-squared:                  0.866
Method:                 Least Squares   F-statistic:                     23.97
Date:                Sun, 15 Aug 2021   Prob (F-statistic):          5.05e-178
Time:                        01:53:07   Log-Likelihood:                -593.89
No. Observations:                 725   AIC:                             1596.
Df Residuals:                     521   BIC:                             2531.
Df Model:                         203                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Analyst
#Treatment Group
#Without CARea
#Score

for i in ['Revision','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Revision ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ NewItems+ POST + POScore',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.409
Model:                            OLS   Adj. R-squared:                  0.117
Method:                 Least Squares   F-statistic:                     1.402
Date:                Sun, 15 Aug 2021   Prob (F-statistic):            0.00421
Time:                        01:53:54   Log-Likelihood:                 618.69
No. Observations:                 527   AIC:                            -887.4
Df Residuals:                     352   BIC:                            -140.6
Df Model:                         174                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
#Analyst
#Treatment Group
#With CARea
#Score
#Car_e0

for i in ['Revision','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Car_e0']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Revision ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ NewItems+ POST + POScore + Car_e0',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.409
Model:                            OLS   Adj. R-squared:                  0.115
Method:                 Least Squares   F-statistic:                     1.390
Date:                Sun, 15 Aug 2021   Prob (F-statistic):            0.00508
Time:                        01:55:36   Log-Likelihood:                 618.69
No. Observations:                 527   AIC:                            -885.4
Df Residuals:                     351   BIC:                            -134.4
Df Model:                         175                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Analyst
#Treatment Group
#With CARea
#Score
#Car_e1

for i in ['Revision','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Car_e1']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Revision ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ NewItems+ POST + POScore + Car_e1',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.409
Model:                            OLS   Adj. R-squared:                  0.115
Method:                 Least Squares   F-statistic:                     1.391
Date:                Sun, 15 Aug 2021   Prob (F-statistic):            0.00502
Time:                        01:56:11   Log-Likelihood:                 618.75
No. Observations:                 527   AIC:                            -885.5
Df Residuals:                     351   BIC:                            -134.5
Df Model:                         175                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Analyst
#Treatment Group
#With CARea
#Score
#Car_e2

for i in ['Revision','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Car_e2']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Revision ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ NewItems+ POST + POScore + Car_e2',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.409
Model:                            OLS   Adj. R-squared:                  0.115
Method:                 Least Squares   F-statistic:                     1.390
Date:                Sun, 15 Aug 2021   Prob (F-statistic):            0.00508
Time:                        01:56:47   Log-Likelihood:                 618.69
No. Observations:                 527   AIC:                            -885.4
Df Residuals:                     351   BIC:                            -134.4
Df Model:                         175                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Analyst
#Treatment Group
#With CARea
#Score
#Car_e3

for i in ['Revision','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Car_e3']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Revision ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ NewItems+ POST + POScore + Car_e3',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':



Second model result

                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.409
Model:                            OLS   Adj. R-squared:                  0.115
Method:                 Least Squares   F-statistic:                     1.391
Date:                Sun, 15 Aug 2021   Prob (F-statistic):            0.00503
Time:                        01:57:19   Log-Likelihood:                 618.74
No. Observations:                 527   AIC:                            -885.5
Df Residuals:                     351   BIC:                            -134.4
Df Model:                         175                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

In [ ]:
#Analyst
#Treatment Group
#With CARea
#Score
#Scar_e0

for i in ['Revision','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Scar_e0']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Revision ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ NewItems+ POST + POScore + Scar_e0',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.409
Model:                            OLS   Adj. R-squared:                  0.115
Method:                 Least Squares   F-statistic:                     1.391
Date:                Sun, 15 Aug 2021   Prob (F-statistic):            0.00503
Time:                        02:00:32   Log-Likelihood:                 618.73
No. Observations:                 527   AIC:                            -885.5
Df Residuals:                     351   BIC:                            -134.4
Df Model:                         175                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Analyst
#Treatment Group
#With CARea
#Score
#Scar_e1

for i in ['Revision','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Scar_e1']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Revision ~ Score+ Size+ C(cusip)+ Filelate+ NewItems+ POST + POScore + Scar_e1',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.409
Model:                            OLS   Adj. R-squared:                  0.115
Method:                 Least Squares   F-statistic:                     1.390
Date:                Sun, 15 Aug 2021   Prob (F-statistic):            0.00507
Time:                        02:01:48   Log-Likelihood:                 618.70
No. Observations:                 527   AIC:                            -885.4
Df Residuals:                     351   BIC:                            -134.4
Df Model:                         175                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Analyst
#Treatment Group
#With CARea
#Score
#Scar_e2

for i in ['Revision','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Scar_e2']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Revision ~ Score+ Size+ C(cusip)+ Filelate+ NewItems+ POST + POScore + Scar_e2',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.409
Model:                            OLS   Adj. R-squared:                  0.115
Method:                 Least Squares   F-statistic:                     1.391
Date:                Sun, 15 Aug 2021   Prob (F-statistic):            0.00503
Time:                        02:02:20   Log-Likelihood:                 618.74
No. Observations:                 527   AIC:                            -885.5
Df Residuals:                     351   BIC:                            -134.4
Df Model:                         175                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Analyst
#Treatment Group
#With CARea
#Score
#Scar_e3

for i in ['Revision','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Scar_e3']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Revision ~ Score+ Size+ C(cusip)+ Filelate+ NewItems+ POST + POScore + Scar_e3',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.409
Model:                            OLS   Adj. R-squared:                  0.115
Method:                 Least Squares   F-statistic:                     1.390
Date:                Sun, 15 Aug 2021   Prob (F-statistic):            0.00506
Time:                        02:02:46   Log-Likelihood:                 618.71
No. Observations:                 527   AIC:                            -885.4
Df Residuals:                     351   BIC:                            -134.4
Df Model:                         175                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Time-series fixed effects(second model)
#Adoption Avoidance Group

import pandas as pd

import statsmodels.formula.api as sm
#import statsmodels.api as sm1

import numpy as np

df = pd.read_excel('/content/Adoption Avoidance Group August 7, 2021.xlsx') 

dfd = df[['Revision','cusip','fyear','Car0','Car1','Car2','Car3','Scar0','Scar1','Scar2','Scar3','Car_e0','Car_e1','Car_e2','Car_e3','Scar_e0', 'Scar_e1', 'Scar_e2','Scar_e3','Other information MD&A Recoded','Score','Raw Score','Size','Filelate','NewItems (excluding 0)','POST']]
#CL: I added all of the relevant data columns
dfd.rename(columns={'Other information MD&A Recoded':'Auditor_OI','NewItems (excluding 0)':'NewItems'},inplace=True)
dfd['year'] = pd.to_datetime(dfd['fyear'],format='%Y')


dfd['POScore'] = dfd['POST'] * dfd['Score']

#Investor
#Without CARea
#Score

for i in ['Car0','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Car0 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())




Second model result

                            OLS Regression Results                            
Dep. Variable:                   Car0   R-squared:                       0.476
Model:                            OLS   Adj. R-squared:                  0.273
Method:                 Least Squares   F-statistic:                     2.351
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           1.15e-07
Time:                        02:16:38   Log-Likelihood:                 457.26
No. Observations:                 328   AIC:                            -730.5
Df Residuals:                     236   BIC:                            -381.6
Df Model:                          91                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

In [ ]:
#Investor
#Adoption Avoidance Group
#Without CARea
#Score
#Car1

for i in ['Car1','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Car1 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                   Car1   R-squared:                       0.509
Model:                            OLS   Adj. R-squared:                  0.320
Method:                 Least Squares   F-statistic:                     2.694
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           8.49e-10
Time:                        02:18:12   Log-Likelihood:                 488.53
No. Observations:                 328   AIC:                            -793.1
Df Residuals:                     236   BIC:                            -444.1
Df Model:                          91                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Adoption Avoidance Group
#Without CARea
#Score
#Car2

for i in ['Car2','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Car2 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                   Car2   R-squared:                       0.472
Model:                            OLS   Adj. R-squared:                  0.268
Method:                 Least Squares   F-statistic:                     2.316
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           1.89e-07
Time:                        02:18:26   Log-Likelihood:                 456.21
No. Observations:                 328   AIC:                            -728.4
Df Residuals:                     236   BIC:                            -379.5
Df Model:                          91                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Adoption Avoidance Group
#Without CARea
#Score
#Car3

for i in ['Car3','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Car3 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                   Car3   R-squared:                       0.507
Model:                            OLS   Adj. R-squared:                  0.317
Method:                 Least Squares   F-statistic:                     2.670
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           1.19e-09
Time:                        02:18:50   Log-Likelihood:                 487.68
No. Observations:                 328   AIC:                            -791.4
Df Residuals:                     236   BIC:                            -442.4
Df Model:                          91                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Adoption Avoidance Group
#Without CARea
#Score
#Scar0

for i in ['Scar0','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Scar0 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                  Scar0   R-squared:                       0.346
Model:                            OLS   Adj. R-squared:                  0.094
Method:                 Least Squares   F-statistic:                     1.374
Date:                Sun, 15 Aug 2021   Prob (F-statistic):             0.0297
Time:                        03:22:29   Log-Likelihood:                -558.90
No. Observations:                 328   AIC:                             1302.
Df Residuals:                     236   BIC:                             1651.
Df Model:                          91                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Adoption Avoidance Group
#Without CARea
#Score
#Scar1

for i in ['Scar1','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Scar1 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                  Scar1   R-squared:                       0.354
Model:                            OLS   Adj. R-squared:                  0.102
Method:                 Least Squares   F-statistic:                     1.403
Date:                Sun, 15 Aug 2021   Prob (F-statistic):             0.0226
Time:                        03:22:33   Log-Likelihood:                -543.55
No. Observations:                 325   AIC:                             1271.
Df Residuals:                     233   BIC:                             1619.
Df Model:                          91                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Adoption Avoidance Group
#Without CARea
#Score
#Scar2

for i in ['Scar2','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Scar2 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                  Scar2   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.095
Method:                 Least Squares   F-statistic:                     1.376
Date:                Sun, 15 Aug 2021   Prob (F-statistic):             0.0293
Time:                        03:22:39   Log-Likelihood:                -561.82
No. Observations:                 328   AIC:                             1308.
Df Residuals:                     236   BIC:                             1657.
Df Model:                          91                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Adoption Avoidance Group
#Without CARea
#Score
#Scar3

for i in ['Scar3','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Scar3 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                  Scar3   R-squared:                       0.353
Model:                            OLS   Adj. R-squared:                  0.103
Method:                 Least Squares   F-statistic:                     1.414
Date:                Sun, 15 Aug 2021   Prob (F-statistic):             0.0199
Time:                        03:22:43   Log-Likelihood:                -551.84
No. Observations:                 328   AIC:                             1288.
Df Residuals:                     236   BIC:                             1637.
Df Model:                          91                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Adoption Avoidance Group
#With CARea
#Score
#Car0 + Car_e0

for i in ['Car0','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Car_e0']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Car0 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Car_e0',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                   Car0   R-squared:                       0.883
Model:                            OLS   Adj. R-squared:                  0.837
Method:                 Least Squares   F-statistic:                     19.20
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           5.86e-72
Time:                        03:22:47   Log-Likelihood:                 702.68
No. Observations:                 328   AIC:                            -1219.
Df Residuals:                     235   BIC:                            -866.6
Df Model:                          92                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Adoption Avoidance Group
#With CARea
#Score
#Car1 + Car_e1

for i in ['Car1','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Car_e1']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Car1 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Car_e1',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                   Car1   R-squared:                       0.868
Model:                            OLS   Adj. R-squared:                  0.816
Method:                 Least Squares   F-statistic:                     16.74
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           3.46e-66
Time:                        03:22:51   Log-Likelihood:                 703.36
No. Observations:                 328   AIC:                            -1221.
Df Residuals:                     235   BIC:                            -868.0
Df Model:                          92                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Adoption Avoidance Group
#With CARea
#Score
#Car2 + Car_e2

for i in ['Car2','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Car_e2']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Car2 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Car_e2',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                   Car2   R-squared:                       0.883
Model:                            OLS   Adj. R-squared:                  0.837
Method:                 Least Squares   F-statistic:                     19.32
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           3.18e-72
Time:                        03:22:53   Log-Likelihood:                 703.70
No. Observations:                 328   AIC:                            -1221.
Df Residuals:                     235   BIC:                            -868.6
Df Model:                          92                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Adoption Avoidance Group
#With CARea
#Score
#Car3 + Car_e3

for i in ['Car3','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Car_e3']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Car3 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Car_e3',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                   Car3   R-squared:                       0.862
Model:                            OLS   Adj. R-squared:                  0.808
Method:                 Least Squares   F-statistic:                     15.99
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           2.85e-64
Time:                        03:22:54   Log-Likelihood:                 696.69
No. Observations:                 328   AIC:                            -1207.
Df Residuals:                     235   BIC:                            -854.6
Df Model:                          92                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Adoption Avoidance Group
#With CARea
#Score
#Scar0 + Scar_e0

for i in ['Scar0','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Scar_e0']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Scar0 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Scar_e0',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                  Scar0   R-squared:                       0.852
Model:                            OLS   Adj. R-squared:                  0.794
Method:                 Least Squares   F-statistic:                     14.70
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           8.04e-61
Time:                        03:22:55   Log-Likelihood:                -315.36
No. Observations:                 328   AIC:                             816.7
Df Residuals:                     235   BIC:                             1169.
Df Model:                          92                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Adoption Avoidance Group
#With CARea
#Score
#Scar1 + Scar_e1

for i in ['Scar1','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Scar_e1']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Scar1 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Scar_e1',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                  Scar1   R-squared:                       0.794
Model:                            OLS   Adj. R-squared:                  0.712
Method:                 Least Squares   F-statistic:                     9.718
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           1.68e-44
Time:                        03:22:57   Log-Likelihood:                -357.84
No. Observations:                 325   AIC:                             901.7
Df Residuals:                     232   BIC:                             1254.
Df Model:                          92                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Adoption Avoidance Group
#With CARea
#Score
#Scar2 + Scar_e2

for i in ['Scar2','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Scar_e2']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Scar2 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Scar_e2',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                  Scar2   R-squared:                       0.854
Model:                            OLS   Adj. R-squared:                  0.797
Method:                 Least Squares   F-statistic:                     14.95
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           1.59e-61
Time:                        03:22:59   Log-Likelihood:                -315.94
No. Observations:                 328   AIC:                             817.9
Df Residuals:                     235   BIC:                             1171.
Df Model:                          92                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Adoption Avoidance Group
#With CARea
#Score
#Scar3 + Scar_e3

for i in ['Scar3','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Scar_e3']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Scar3 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Scar_e3',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                  Scar3   R-squared:                       0.791
Model:                            OLS   Adj. R-squared:                  0.709
Method:                 Least Squares   F-statistic:                     9.660
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           1.24e-44
Time:                        03:23:01   Log-Likelihood:                -366.58
No. Observations:                 328   AIC:                             919.2
Df Residuals:                     235   BIC:                             1272.
Df Model:                          92                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Analyst
#Adoption Avoidance Group
#Without CARea
#Score


for i in ['Revision','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Revision ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.566
Model:                            OLS   Adj. R-squared:                  0.327
Method:                 Least Squares   F-statistic:                     2.373
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           1.79e-06
Time:                        03:23:12   Log-Likelihood:                 335.36
No. Observations:                 238   AIC:                            -500.7
Df Residuals:                     153   BIC:                            -205.6
Df Model:                          84                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Analyst
#Adoption Avoidance Group
#With CARea
#Score
#Car_e0

for i in ['Revision','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Car_e0']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Revision ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Car_e0',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.571
Model:                            OLS   Adj. R-squared:                  0.331
Method:                 Least Squares   F-statistic:                     2.378
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           1.66e-06
Time:                        03:23:19   Log-Likelihood:                 336.74
No. Observations:                 238   AIC:                            -501.5
Df Residuals:                     152   BIC:                            -202.9
Df Model:                          85                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Analyst
#Adoption Avoidance Group
#With CARea
#Score
#Car_e1

for i in ['Revision','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Car_e1']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Revision ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Car_e1',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.569
Model:                            OLS   Adj. R-squared:                  0.328
Method:                 Least Squares   F-statistic:                     2.364
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           1.95e-06
Time:                        03:23:20   Log-Likelihood:                 336.32
No. Observations:                 238   AIC:                            -500.6
Df Residuals:                     152   BIC:                            -202.0
Df Model:                          85                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Analyst
#Adoption Avoidance Group
#With CARea
#Score
#Car_e2

for i in ['Revision','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Car_e2']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Revision ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Car_e2',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.571
Model:                            OLS   Adj. R-squared:                  0.331
Method:                 Least Squares   F-statistic:                     2.382
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           1.59e-06
Time:                        03:23:22   Log-Likelihood:                 336.85
No. Observations:                 238   AIC:                            -501.7
Df Residuals:                     152   BIC:                            -203.1
Df Model:                          85                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Analyst
#Adoption Avoidance Group
#With CARea
#Score
#Car_e3

for i in ['Revision','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Car_e3']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Revision ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Car_e3',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.569
Model:                            OLS   Adj. R-squared:                  0.329
Method:                 Least Squares   F-statistic:                     2.364
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           1.93e-06
Time:                        03:23:24   Log-Likelihood:                 336.34
No. Observations:                 238   AIC:                            -500.7
Df Residuals:                     152   BIC:                            -202.1
Df Model:                          85                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Analyst
#Adoption Avoidance Group
#With CARea
#Score
#Scar_e0

for i in ['Revision','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Scar_e0']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Revision ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Scar_e0',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.566
Model:                            OLS   Adj. R-squared:                  0.323
Method:                 Least Squares   F-statistic:                     2.331
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           2.78e-06
Time:                        03:23:25   Log-Likelihood:                 335.38
No. Observations:                 238   AIC:                            -498.8
Df Residuals:                     152   BIC:                            -200.2
Df Model:                          85                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Analyst
#Adoption Avoidance Group
#With CARea
#Score
#Scar_e1

for i in ['Revision','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Scar_e1']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Revision ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Scar_e1',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.566
Model:                            OLS   Adj. R-squared:                  0.319
Method:                 Least Squares   F-statistic:                     2.287
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           5.03e-06
Time:                        03:23:27   Log-Likelihood:                 329.86
No. Observations:                 235   AIC:                            -487.7
Df Residuals:                     149   BIC:                            -190.2
Df Model:                          85                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Analyst
#Adoption Avoidance Group
#With CARea
#Score
#Scar_e2

for i in ['Revision','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Scar_e2']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Revision ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Scar_e2',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.566
Model:                            OLS   Adj. R-squared:                  0.323
Method:                 Least Squares   F-statistic:                     2.332
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           2.75e-06
Time:                        03:23:28   Log-Likelihood:                 335.41
No. Observations:                 238   AIC:                            -498.8
Df Residuals:                     152   BIC:                            -200.2
Df Model:                          85                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Analyst
#Adoption Avoidance Group
#With CARea
#Score
#Scar_e3

for i in ['Revision','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Scar_e3']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Revision ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Scar_e3',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.566
Model:                            OLS   Adj. R-squared:                  0.323
Method:                 Least Squares   F-statistic:                     2.330
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           2.81e-06
Time:                        03:23:32   Log-Likelihood:                 335.36
No. Observations:                 238   AIC:                            -498.7
Df Residuals:                     152   BIC:                            -200.1
Df Model:                          85                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Time-series fixed effects(second model)
#Non-Adoption Group

import pandas as pd

import statsmodels.formula.api as sm
#import statsmodels.api as sm1

import numpy as np

df = pd.read_excel('/content/Non-Adoption Group Data August 7, 2021.xlsx') 

dfd = df[['Revision','cusip','fyear','Car0','Car1','Car2','Car3','Scar0','Scar1','Scar2','Scar3','Car_e0','Car_e1','Car_e2','Car_e3','Scar_e0', 'Scar_e1', 'Scar_e2','Scar_e3','Other information MD&A Recoded','Score','Raw Score','Size','Filelate','NewItems (excluding 0)','POST']]
#CL: I added all of the relevant data columns
dfd.rename(columns={'Other information MD&A Recoded':'Auditor_OI','NewItems (excluding 0)':'NewItems'},inplace=True)
dfd['year'] = pd.to_datetime(dfd['fyear'],format='%Y')


dfd['POScore'] = dfd['POST'] * dfd['Score']

#Investor
#Without CARea
#Score

for i in ['Car0','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Car0 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())




Second model result

                            OLS Regression Results                            
Dep. Variable:                   Car0   R-squared:                       0.683
Model:                            OLS   Adj. R-squared:                  0.518
Method:                 Least Squares   F-statistic:                     4.140
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           8.18e-06
Time:                        03:23:45   Log-Likelihood:                 158.08
No. Observations:                  77   AIC:                            -262.2
Df Residuals:                      50   BIC:                            -198.9
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

In [ ]:
#Investor
#Non-Adoption Group
#Without CARea
#Score
#Car1

for i in ['Car1','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Car1 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                   Car1   R-squared:                       0.630
Model:                            OLS   Adj. R-squared:                  0.438
Method:                 Least Squares   F-statistic:                     3.280
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           0.000154
Time:                        03:23:58   Log-Likelihood:                 165.41
No. Observations:                  77   AIC:                            -276.8
Df Residuals:                      50   BIC:                            -213.5
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Non-Adoption Group
#Without CARea
#Score
#Car2

for i in ['Car2','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Car2 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                   Car2   R-squared:                       0.701
Model:                            OLS   Adj. R-squared:                  0.545
Method:                 Least Squares   F-statistic:                     4.500
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           2.58e-06
Time:                        03:23:59   Log-Likelihood:                 162.17
No. Observations:                  77   AIC:                            -270.3
Df Residuals:                      50   BIC:                            -207.1
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Non-Adoption Group
#Without CARea
#Score
#Car3

for i in ['Car3','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Car3 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                   Car3   R-squared:                       0.633
Model:                            OLS   Adj. R-squared:                  0.442
Method:                 Least Squares   F-statistic:                     3.315
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           0.000136
Time:                        03:24:01   Log-Likelihood:                 168.53
No. Observations:                  77   AIC:                            -283.1
Df Residuals:                      50   BIC:                            -219.8
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Non-Adoption Group
#Without CARea
#Score
#Scar0

for i in ['Scar0','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Scar0 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                  Scar0   R-squared:                       0.421
Model:                            OLS   Adj. R-squared:                  0.120
Method:                 Least Squares   F-statistic:                     1.398
Date:                Sun, 15 Aug 2021   Prob (F-statistic):              0.153
Time:                        03:24:02   Log-Likelihood:                -107.35
No. Observations:                  77   AIC:                             268.7
Df Residuals:                      50   BIC:                             332.0
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Non-Adoption Group
#Without CARea
#Score
#Scar1

for i in ['Scar1','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Scar1 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                  Scar1   R-squared:                       0.381
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     1.182
Date:                Sun, 15 Aug 2021   Prob (F-statistic):              0.300
Time:                        03:24:04   Log-Likelihood:                -106.28
No. Observations:                  77   AIC:                             266.6
Df Residuals:                      50   BIC:                             329.8
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Non-Adoption Group
#Without CARea
#Score
#Scar2

for i in ['Scar2','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Scar2 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                  Scar2   R-squared:                       0.466
Model:                            OLS   Adj. R-squared:                  0.188
Method:                 Least Squares   F-statistic:                     1.677
Date:                Sun, 15 Aug 2021   Prob (F-statistic):             0.0580
Time:                        03:24:05   Log-Likelihood:                -104.30
No. Observations:                  77   AIC:                             262.6
Df Residuals:                      50   BIC:                             325.9
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Non-Adoption Group
#Without CARea
#Score
#Scar3

for i in ['Scar3','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Scar3 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                  Scar3   R-squared:                       0.399
Model:                            OLS   Adj. R-squared:                  0.087
Method:                 Least Squares   F-statistic:                     1.278
Date:                Sun, 15 Aug 2021   Prob (F-statistic):              0.225
Time:                        03:24:07   Log-Likelihood:                -105.25
No. Observations:                  77   AIC:                             264.5
Df Residuals:                      50   BIC:                             327.8
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Adoption Group
#With CARea
#Score
#Car0 + Car_e0

for i in ['Car0','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Car_e0']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Car0 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Car_e0',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                   Car0   R-squared:                       0.752
Model:                            OLS   Adj. R-squared:                  0.615
Method:                 Least Squares   F-statistic:                     5.502
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           1.29e-07
Time:                        03:24:08   Log-Likelihood:                 167.54
No. Observations:                  77   AIC:                            -279.1
Df Residuals:                      49   BIC:                            -213.5
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Non-Adoption Group
#With CARea
#Score
#Car1 + Car_e1

for i in ['Car1','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Car_e1']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Car1 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Car_e1',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                   Car1   R-squared:                       0.765
Model:                            OLS   Adj. R-squared:                  0.636
Method:                 Least Squares   F-statistic:                     5.922
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           4.04e-08
Time:                        03:24:11   Log-Likelihood:                 182.91
No. Observations:                  77   AIC:                            -309.8
Df Residuals:                      49   BIC:                            -244.2
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Non-Adoption Group
#With CARea
#Score
#Car2 + Car_e2

for i in ['Car2','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Car_e2']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Car2 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Car_e2',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                   Car2   R-squared:                       0.753
Model:                            OLS   Adj. R-squared:                  0.616
Method:                 Least Squares   F-statistic:                     5.520
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           1.22e-07
Time:                        03:24:12   Log-Likelihood:                 169.51
No. Observations:                  77   AIC:                            -283.0
Df Residuals:                      49   BIC:                            -217.4
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Non-Adoption Group
#With CARea
#Score
#Car3 + Car_e3

for i in ['Car3','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Car_e3']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Car3 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Car_e3',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                   Car3   R-squared:                       0.749
Model:                            OLS   Adj. R-squared:                  0.611
Method:                 Least Squares   F-statistic:                     5.422
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           1.61e-07
Time:                        03:24:14   Log-Likelihood:                 183.20
No. Observations:                  77   AIC:                            -310.4
Df Residuals:                      49   BIC:                            -244.8
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Non-Adoption Group
#With CARea
#Score
#Scar0 + Scar_e0

for i in ['Scar0','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Scar_e0']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Scar0 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Scar_e0',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                  Scar0   R-squared:                       0.524
Model:                            OLS   Adj. R-squared:                  0.261
Method:                 Least Squares   F-statistic:                     1.996
Date:                Sun, 15 Aug 2021   Prob (F-statistic):             0.0175
Time:                        03:24:16   Log-Likelihood:                -99.822
No. Observations:                  77   AIC:                             255.6
Df Residuals:                      49   BIC:                             321.3
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Non-Adoption Group
#With CARea
#Score
#Scar1 + Scar_e1

for i in ['Scar1','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Scar_e1']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Scar1 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Scar_e1',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                  Scar1   R-squared:                       0.546
Model:                            OLS   Adj. R-squared:                  0.295
Method:                 Least Squares   F-statistic:                     2.179
Date:                Sun, 15 Aug 2021   Prob (F-statistic):            0.00878
Time:                        03:24:19   Log-Likelihood:                -94.357
No. Observations:                  77   AIC:                             244.7
Df Residuals:                      49   BIC:                             310.3
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Non-Adoption Group
#With CARea
#Score
#Scar2 + Scar_e2

for i in ['Scar2','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Scar_e2']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Scar2 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Scar_e2',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                  Scar2   R-squared:                       0.547
Model:                            OLS   Adj. R-squared:                  0.297
Method:                 Least Squares   F-statistic:                     2.190
Date:                Sun, 15 Aug 2021   Prob (F-statistic):            0.00841
Time:                        03:24:21   Log-Likelihood:                -97.969
No. Observations:                  77   AIC:                             251.9
Df Residuals:                      49   BIC:                             317.6
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Investor
#Non-Adoption Group
#With CARea
#Score
#Scar3 + Scar_e3

for i in ['Scar3','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Scar_e3']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Scar3 ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Scar_e3',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:                  Scar3   R-squared:                       0.538
Model:                            OLS   Adj. R-squared:                  0.284
Method:                 Least Squares   F-statistic:                     2.116
Date:                Sun, 15 Aug 2021   Prob (F-statistic):             0.0111
Time:                        03:24:22   Log-Likelihood:                -95.107
No. Observations:                  77   AIC:                             246.2
Df Residuals:                      49   BIC:                             311.8
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Analyst
#Non-Adoption Group
#Without CARea
#Score


for i in ['Revision','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Revision ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.720
Model:                            OLS   Adj. R-squared:                  0.538
Method:                 Least Squares   F-statistic:                     3.961
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           8.89e-05
Time:                        03:24:24   Log-Likelihood:                 118.64
No. Observations:                  62   AIC:                            -187.3
Df Residuals:                      37   BIC:                            -134.1
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Analyst
#Non-Adoption Group
#With CARea
#Score
#Car_e0

for i in ['Revision','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Car_e0']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Revision ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Car_e0',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.773
Model:                            OLS   Adj. R-squared:                  0.611
Method:                 Least Squares   F-statistic:                     4.773
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           1.44e-05
Time:                        03:24:30   Log-Likelihood:                 122.68
No. Observations:                  61   AIC:                            -193.4
Df Residuals:                      35   BIC:                            -138.5
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Analyst
#Non-Adoption Group
#With CARea
#Score
#Car_e1

for i in ['Revision','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Car_e1']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Revision ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Car_e1',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.777
Model:                            OLS   Adj. R-squared:                  0.618
Method:                 Least Squares   F-statistic:                     4.886
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           1.11e-05
Time:                        03:24:33   Log-Likelihood:                 123.23
No. Observations:                  61   AIC:                            -194.5
Df Residuals:                      35   BIC:                            -139.6
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Analyst
#Non-Adoption Group
#With CARea
#Score
#Car_e2

for i in ['Revision','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Car_e2']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Revision ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Car_e2',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.765
Model:                            OLS   Adj. R-squared:                  0.598
Method:                 Least Squares   F-statistic:                     4.567
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           2.34e-05
Time:                        03:24:34   Log-Likelihood:                 121.64
No. Observations:                  61   AIC:                            -191.3
Df Residuals:                      35   BIC:                            -136.4
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Analyst
#Non-Adoption Group
#With CARea
#Score
#Car_e3

for i in ['Revision','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Car_e3']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Revision ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Car_e3',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.773
Model:                            OLS   Adj. R-squared:                  0.610
Method:                 Least Squares   F-statistic:                     4.755
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           1.50e-05
Time:                        03:24:35   Log-Likelihood:                 122.59
No. Observations:                  61   AIC:                            -193.2
Df Residuals:                      35   BIC:                            -138.3
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Analyst
#Non-Adoption Group
#With CARea
#Score
#Scar_e0

for i in ['Revision','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Scar_e0']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Revision ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Scar_e0',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.726
Model:                            OLS   Adj. R-squared:                  0.530
Method:                 Least Squares   F-statistic:                     3.701
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           0.000208
Time:                        03:24:37   Log-Likelihood:                 116.86
No. Observations:                  61   AIC:                            -181.7
Df Residuals:                      35   BIC:                            -126.8
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Analyst
#Non-Adoption Group
#With CARea
#Score
#Scar_e1

for i in ['Revision','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Scar_e1']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Revision ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Scar_e1',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.726
Model:                            OLS   Adj. R-squared:                  0.530
Method:                 Least Squares   F-statistic:                     3.706
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           0.000205
Time:                        03:24:39   Log-Likelihood:                 116.89
No. Observations:                  61   AIC:                            -181.8
Df Residuals:                      35   BIC:                            -126.9
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Analyst
#Non-Adoption Group
#With CARea
#Score
#Scar_e2

for i in ['Revision','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Scar_e2']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Revision ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Scar_e2',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.724
Model:                            OLS   Adj. R-squared:                  0.526
Method:                 Least Squares   F-statistic:                     3.664
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           0.000229
Time:                        03:24:40   Log-Likelihood:                 116.63
No. Observations:                  61   AIC:                            -181.3
Df Residuals:                      35   BIC:                            -126.4
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Analyst
#Non-Adoption Group
#With CARea
#Score
#Scar_e3

for i in ['Revision','Score','Size','Filelate' ,'NewItems', 'POST', 'POScore','Scar_e3']:
  mean = dfd.groupby('year')[i].transform(np.mean)
  dfd[i] = dfd[i] - mean

ols2 = sm.ols(formula='Revision ~ Score+ Size+ NewItems+ C(cusip)+ Filelate+ POST + POScore + Scar_e3',
                          data=dfd).fit()
#C(cusip) is the company fixed effects
print('\nSecond model result\n')

print(ols2.summary())


Second model result

                            OLS Regression Results                            
Dep. Variable:               Revision   R-squared:                       0.724
Model:                            OLS   Adj. R-squared:                  0.527
Method:                 Least Squares   F-statistic:                     3.672
Date:                Sun, 15 Aug 2021   Prob (F-statistic):           0.000224
Time:                        03:24:41   Log-Likelihood:                 116.69
No. Observations:                  61   AIC:                            -181.4
Df Residuals:                      35   BIC:                            -126.5
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Other Formulas for statsmodels:

#dfdd = dfd.dropna(subset=['Car_e1','Score','Size','Filelate' ,'NewItems', 'Auditor_OI', 'AuScore'])
#dfdd = dfd.dropna()


In [ ]:
dfd['cik']

0        2809
1        2809
2        2809
3        2809
4      885590
       ...   
88    1318220
89    1318220
90    1318220
91    1318220
92    1318220
Name: cik, Length: 93, dtype: int64

In [ ]:
pip install linearmodels

     |████████████████████████████████| 1.5 MB 5.3 MB/s 
     |████████████████████████████████| 9.5 MB 20.5 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [ ]:

import pandas as pd
from linearmodels.panel import PanelOLS
from statsmodels.tools.tools import add_constant

import numpy as np

df = pd.read_excel('/content/Treatment Group August 7, 2021.xlsx') 

#Alternative for fixed effect time-series
dfd = df[['Revision','cusip','fyear','Car0','Car1','Car2','Car3','Scar0','Scar1','Scar2','Scar3','Car_e0','Car_e1','Car_e2','Car_e3','Scar_e0', 'Scar_e1', 'Scar_e2','Scar_e3','Other information MD&A Recoded','Score','Raw Score','Size','Filelate','NewItems (excluding 0)','POST']]

dfd.rename(columns={'Other information MD&A Recoded':'Auditor_OI','NewItems (excluding 0)':'NewItems'},inplace=True)

dfd['year'] = dfd['fyear'].astype(int)
#dfd['year'] = pd.Categorical(dfd.year)
dfd['year'] = pd.to_datetime(dfd.year, format='%Y')
dfd['cusip'] = pd.Categorical(dfd['cusip'])
dfd = dfd.set_index(['cusip','year'])
dfd['POScore'] = dfd['POST'] * dfd['Score']

#Car0
#Score

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore']])

FE = PanelOLS(dfd.Car0, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

Dep. Variable:,Car0,R-squared:,0.0153
Estimator:,PanelOLS,R-squared (Between):,-0.6049
No. Observations:,725,R-squared (Within):,-0.0179
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.2042
Time:,18:47:33,Log-likelihood,1203.7
Cov. Estimator:,Clustered,,
,,F-statistic:,1.3414
Entities:,198,P-value,0.2368
Avg Obs:,3.6616,Distribution:,"F(6,519)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,6.2455


In [ ]:
#Investor
#Without CARea
#Treatment Group
#Score
#Car1

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore']])

FE = PanelOLS(dfd.Car1, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )
result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Car1,R-squared:,0.0255
Estimator:,PanelOLS,R-squared (Between):,-1.9127
No. Observations:,725,R-squared (Within):,-0.0502
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.6726
Time:,00:23:24,Log-likelihood,1234.0
Cov. Estimator:,Clustered,,
,,F-statistic:,2.2632
Entities:,198,P-value,0.0363
Avg Obs:,3.6616,Distribution:,"F(6,519)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,0.3441


In [ ]:
#Investor
#Without CARea
#Treatment Group
#Score
#Car2

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore']])

FE = PanelOLS(dfd.Car2, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )
result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Car2,R-squared:,0.0151
Estimator:,PanelOLS,R-squared (Between):,-0.5874
No. Observations:,725,R-squared (Within):,-0.0170
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.1979
Time:,01:20:44,Log-likelihood,1202.4
Cov. Estimator:,Clustered,,
,,F-statistic:,1.3236
Entities:,198,P-value,0.2447
Avg Obs:,3.6616,Distribution:,"F(6,519)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,6.2119


In [ ]:
#Investor
#Without CARea
#Treatment Group
#Score
#Car3

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore']])

FE = PanelOLS(dfd.Car3, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )
result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Car3,R-squared:,0.0252
Estimator:,PanelOLS,R-squared (Between):,-1.8637
No. Observations:,725,R-squared (Within):,-0.0537
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.6617
Time:,18:47:53,Log-likelihood,1235.8
Cov. Estimator:,Clustered,,
,,F-statistic:,2.2406
Entities:,198,P-value,0.0382
Avg Obs:,3.6616,Distribution:,"F(6,519)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,0.1486


In [ ]:
#Investor
#Without CARea
#Treatment Group
#Score
#Scar0

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore']])

FE = PanelOLS(dfd.Scar0, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )
result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Scar0,R-squared:,0.0060
Estimator:,PanelOLS,R-squared (Between):,-0.5475
No. Observations:,725,R-squared (Within):,0.0193
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.1776
Time:,18:50:24,Log-likelihood,-1282.0
Cov. Estimator:,Clustered,,
,,F-statistic:,0.5216
Entities:,198,P-value,0.7921
Avg Obs:,3.6616,Distribution:,"F(6,519)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,1.2232


In [ ]:
#Investor
#Without CARea
#Treatment Group
#Score
#Scar1

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore']])

FE = PanelOLS(dfd.Scar1, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )
result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Scar1,R-squared:,0.0128
Estimator:,PanelOLS,R-squared (Between):,-1.7253
No. Observations:,725,R-squared (Within):,0.0189
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.5719
Time:,18:50:14,Log-likelihood,-1301.9
Cov. Estimator:,Clustered,,
,,F-statistic:,1.1196
Entities:,198,P-value,0.3495
Avg Obs:,3.6616,Distribution:,"F(6,519)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,2.9673


In [ ]:
#Investor
#Without CARea
#Treatment Group
#Score
#Scar2

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore']])

FE = PanelOLS(dfd.Scar2, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )
result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Scar2,R-squared:,0.0058
Estimator:,PanelOLS,R-squared (Between):,-0.4895
No. Observations:,725,R-squared (Within):,0.0186
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.1616
Time:,18:49:56,Log-likelihood,-1287.3
Cov. Estimator:,Clustered,,
,,F-statistic:,0.5045
Entities:,198,P-value,0.8051
Avg Obs:,3.6616,Distribution:,"F(6,519)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,1.1054


In [ ]:
#Investor
#Without CARea
#Treatment Group
#Score
#Scar3

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore']])

FE = PanelOLS(dfd.Scar3, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )
result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Scar3,R-squared:,0.0125
Estimator:,PanelOLS,R-squared (Between):,-1.6059
No. Observations:,725,R-squared (Within):,0.0169
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.5418
Time:,18:51:18,Log-likelihood,-1304.9
Cov. Estimator:,Clustered,,
,,F-statistic:,1.0928
Entities:,198,P-value,0.3654
Avg Obs:,3.6616,Distribution:,"F(6,519)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,2.9209


In [ ]:
import pandas as pd
from linearmodels.panel import PanelOLS
from statsmodels.tools.tools import add_constant

import numpy as np

df = pd.read_excel('/content/Treatment Group August 7, 2021.xlsx') 

#Alternative for fixed effect time-series
dfd = df[['Revision','cusip','fyear','Car0','Car1','Car2','Car3','Scar0','Scar1','Scar2','Scar3','Car_e0','Car_e1','Car_e2','Car_e3','Scar_e0', 'Scar_e1', 'Scar_e2','Scar_e3','Other information MD&A Recoded','Score','Raw Score','Size','Filelate','NewItems (excluding 0)','POST']]

dfd.rename(columns={'Other information MD&A Recoded':'Auditor_OI','NewItems (excluding 0)':'NewItems'},inplace=True)

dfd['year'] = dfd['fyear'].astype(int)
#dfd['year'] = pd.Categorical(dfd.year)
dfd['year'] = pd.to_datetime(dfd.year, format='%Y')
dfd['cusip'] = pd.Categorical(dfd['cusip'])
dfd = dfd.set_index(['cusip','year'])
dfd['POScore'] = dfd['POST'] * dfd['Score']

#Score
#CAR0 and CAR_e0
exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Car_e0']])

FE = PanelOLS(dfd.Car0, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

Dep. Variable:,Car0,R-squared:,0.8878
Estimator:,PanelOLS,R-squared (Between):,0.5293
No. Observations:,725,R-squared (Within):,0.8932
Date:,"Sat, Aug 14 2021",R-squared (Overall):,0.7741
Time:,18:52:02,Log-likelihood,1991.0
Cov. Estimator:,Clustered,,
,,F-statistic:,585.31
Entities:,198,P-value,0.0000
Avg Obs:,3.6616,Distribution:,"F(7,518)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,2005.1


In [ ]:
#Investor
#With CARea
#Treatment Group
#Score
#CAR1 and CAR_e1
exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Car_e1']])

FE = PanelOLS(dfd.Car1, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Car1,R-squared:,0.8565
Estimator:,PanelOLS,R-squared (Between):,0.3593
No. Observations:,725,R-squared (Within):,0.8655
Date:,"Fri, Aug 13 2021",R-squared (Overall):,0.6927
Time:,21:30:08,Log-likelihood,1928.3
Cov. Estimator:,Clustered,,
,,F-statistic:,441.62
Entities:,198,P-value,0.0000
Avg Obs:,3.6616,Distribution:,"F(7,518)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,825.86


In [ ]:
#Investor
#With CARea
#Treatment Group
#Score
#CAR2 and CAR_e2
exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Car_e2']])

FE = PanelOLS(dfd.Car2, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Car2,R-squared:,0.8882
Estimator:,PanelOLS,R-squared (Between):,0.5281
No. Observations:,725,R-squared (Within):,0.8934
Date:,"Fri, Aug 13 2021",R-squared (Overall):,0.7739
Time:,21:31:29,Log-likelihood,1991.3
Cov. Estimator:,Clustered,,
,,F-statistic:,588.02
Entities:,198,P-value,0.0000
Avg Obs:,3.6616,Distribution:,"F(7,518)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,802.87


In [ ]:
#Investor
#With CARea
#Treatment Group
#Score
#CAR3 and CAR_e3
exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Car_e3']])

FE = PanelOLS(dfd.Car3, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Car3,R-squared:,0.8534
Estimator:,PanelOLS,R-squared (Between):,0.3675
No. Observations:,725,R-squared (Within):,0.8628
Date:,"Sat, Aug 14 2021",R-squared (Overall):,0.6926
Time:,18:52:45,Log-likelihood,1922.5
Cov. Estimator:,Clustered,,
,,F-statistic:,430.63
Entities:,198,P-value,0.0000
Avg Obs:,3.6616,Distribution:,"F(7,518)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,762.05


In [ ]:
#Investor
#With CARea
#Treatment Group
#Score
#Scar0 and Scar_e0
exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Scar_e0']])

FE = PanelOLS(dfd.Scar0, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Scar0,R-squared:,0.8917
Estimator:,PanelOLS,R-squared (Between):,0.5456
No. Observations:,725,R-squared (Within):,0.8935
Date:,"Sat, Aug 14 2021",R-squared (Overall):,0.7742
Time:,18:54:10,Log-likelihood,-478.52
Cov. Estimator:,Clustered,,
,,F-statistic:,609.11
Entities:,198,P-value,0.0000
Avg Obs:,3.6616,Distribution:,"F(7,518)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,742.69


In [ ]:
#Investor
#With CARea
#Treatment Group
#Score
#Scar1 and Scar_e1
exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Scar_e1']])

FE = PanelOLS(dfd.Scar1, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Scar1,R-squared:,0.8638
Estimator:,PanelOLS,R-squared (Between):,0.5303
No. Observations:,725,R-squared (Within):,0.8700
Date:,"Sat, Aug 14 2021",R-squared (Overall):,0.7498
Time:,18:54:43,Log-likelihood,-583.79
Cov. Estimator:,Clustered,,
,,F-statistic:,469.49
Entities:,198,P-value,0.0000
Avg Obs:,3.6616,Distribution:,"F(7,518)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,983.05


In [ ]:
#Investor
#With CARea
#Treatment Group
#Score
#Scar2 and Scar_e2

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Scar_e2']])

FE = PanelOLS(dfd.Scar2, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Scar2,R-squared:,0.8917
Estimator:,PanelOLS,R-squared (Between):,0.5524
No. Observations:,725,R-squared (Within):,0.8936
Date:,"Sat, Aug 14 2021",R-squared (Overall):,0.7754
Time:,19:36:08,Log-likelihood,-483.55
Cov. Estimator:,Clustered,,
,,F-statistic:,609.42
Entities:,198,P-value,0.0000
Avg Obs:,3.6616,Distribution:,"F(7,518)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,770.56


In [ ]:
#Investor
#With CARea
#Treatment Group
#Scar3 and Scar_e3
#Score

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Scar_e3']])

FE = PanelOLS(dfd.Scar3, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Scar3,R-squared:,0.8612
Estimator:,PanelOLS,R-squared (Between):,0.5385
No. Observations:,725,R-squared (Within):,0.8679
Date:,"Sat, Aug 14 2021",R-squared (Overall):,0.7495
Time:,19:36:37,Log-likelihood,-593.76
Cov. Estimator:,Clustered,,
,,F-statistic:,458.98
Entities:,198,P-value,0.0000
Avg Obs:,3.6616,Distribution:,"F(7,518)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,958.17


In [ ]:
#Analyst
#Without CARea
#Treatment Group
#Score

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore']])

FE = PanelOLS(dfd.Revision, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Revision,R-squared:,0.0485
Estimator:,PanelOLS,R-squared (Between):,-9.8520
No. Observations:,527,R-squared (Within):,-0.0478
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-4.2191
Time:,19:40:02,Log-likelihood,618.78
Cov. Estimator:,Clustered,,
,,F-statistic:,2.9639
Entities:,198,P-value,0.0078
Avg Obs:,2.6616,Distribution:,"F(6,349)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,5.5480


In [ ]:
#Analyst
#With CARea
#Treatment Group
#Score
#Car_e0

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Car_e0']])

FE = PanelOLS(dfd.Revision, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Revision,R-squared:,0.0485
Estimator:,PanelOLS,R-squared (Between):,-9.8495
No. Observations:,527,R-squared (Within):,-0.0479
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-4.2180
Time:,19:42:04,Log-likelihood,618.78
Cov. Estimator:,Clustered,,
,,F-statistic:,2.5332
Entities:,198,P-value,0.0149
Avg Obs:,2.6616,Distribution:,"F(7,348)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,1.6009


In [ ]:
#Analyst
#With CARea
#Treatment Group
#Score
#Car_e1

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Car_e1']])

FE = PanelOLS(dfd.Revision, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Revision,R-squared:,0.0487
Estimator:,PanelOLS,R-squared (Between):,-10.031
No. Observations:,527,R-squared (Within):,-0.0462
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-4.2983
Time:,19:42:24,Log-likelihood,618.85
Cov. Estimator:,Clustered,,
,,F-statistic:,2.5457
Entities:,198,P-value,0.0144
Avg Obs:,2.6616,Distribution:,"F(7,348)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,2.1670


In [ ]:
#Analyst
#With CARea
#Treatment Group
#Score
#Car_e2

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Car_e2']])

FE = PanelOLS(dfd.Revision, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Revision,R-squared:,0.0485
Estimator:,PanelOLS,R-squared (Between):,-9.8472
No. Observations:,527,R-squared (Within):,-0.0479
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-4.2169
Time:,19:42:39,Log-likelihood,618.78
Cov. Estimator:,Clustered,,
,,F-statistic:,2.5333
Entities:,198,P-value,0.0149
Avg Obs:,2.6616,Distribution:,"F(7,348)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,1.5875


In [ ]:
#Analyst
#With CARea
#Treatment Group
#Score
#Car_e3

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Car_e3']])

FE = PanelOLS(dfd.Revision, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Revision,R-squared:,0.0487
Estimator:,PanelOLS,R-squared (Between):,-10.013
No. Observations:,527,R-squared (Within):,-0.0464
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-4.2906
Time:,19:42:53,Log-likelihood,618.83
Cov. Estimator:,Clustered,,
,,F-statistic:,2.5435
Entities:,198,P-value,0.0145
Avg Obs:,2.6616,Distribution:,"F(7,348)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,2.1271


In [ ]:
#Analyst
#With CARea
#Treatment Group
#Score
#Scar_e0

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Scar_e0']])

FE = PanelOLS(dfd.Revision, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Revision,R-squared:,0.0486
Estimator:,PanelOLS,R-squared (Between):,-9.7523
No. Observations:,527,R-squared (Within):,-0.0469
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-4.1734
Time:,19:50:35,Log-likelihood,618.82
Cov. Estimator:,Clustered,,
,,F-statistic:,2.5415
Entities:,198,P-value,0.0146
Avg Obs:,2.6616,Distribution:,"F(7,348)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,0.1354


In [ ]:
#Analyst
#With CARea
#Treatment Group
#Score
#Scar_e1

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Scar_e1']])

FE = PanelOLS(dfd.Revision, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Revision,R-squared:,0.0485
Estimator:,PanelOLS,R-squared (Between):,-9.7744
No. Observations:,527,R-squared (Within):,-0.0478
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-4.1845
Time:,19:50:57,Log-likelihood,618.79
Cov. Estimator:,Clustered,,
,,F-statistic:,2.5348
Entities:,198,P-value,0.0148
Avg Obs:,2.6616,Distribution:,"F(7,348)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,-1.6506


In [ ]:
#Analyst
#With CARea
#Treatment Group
#Score
#Scar_e2

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Scar_e2']])

FE = PanelOLS(dfd.Revision, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Revision,R-squared:,0.0486
Estimator:,PanelOLS,R-squared (Between):,-9.7488
No. Observations:,527,R-squared (Within):,-0.0468
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-4.1717
Time:,19:51:10,Log-likelihood,618.83
Cov. Estimator:,Clustered,,
,,F-statistic:,2.5421
Entities:,198,P-value,0.0146
Avg Obs:,2.6616,Distribution:,"F(7,348)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,0.0198


In [ ]:
#Analyst
#With CARea
#Treatment Group
#Score
#Scar_e3

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Scar_e3']])

FE = PanelOLS(dfd.Revision, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Revision,R-squared:,0.0485
Estimator:,PanelOLS,R-squared (Between):,-9.7542
No. Observations:,527,R-squared (Within):,-0.0478
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-4.1755
Time:,19:51:22,Log-likelihood,618.80
Cov. Estimator:,Clustered,,
,,F-statistic:,2.5357
Entities:,198,P-value,0.0148
Avg Obs:,2.6616,Distribution:,"F(7,348)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,-1.9373


In [ ]:
#Adoption avoidance group
#Score
import pandas as pd
from linearmodels.panel import PanelOLS
from statsmodels.tools.tools import add_constant

import numpy as np

df = pd.read_excel('/content/Adoption Avoidance Group August 7, 2021.xlsx') 

#Alternative for fixed effect time-series
dfd = df[['Revision','cusip','fyear','Car0','Car1','Car2','Car3','Scar0','Scar1','Scar2','Scar3','Car_e0','Car_e1','Car_e2','Car_e3','Scar_e0', 'Scar_e1', 'Scar_e2','Scar_e3','Other information MD&A Recoded','Score','Raw Score','Size','Filelate','NewItems (excluding 0)','POST']]

dfd.rename(columns={'Other information MD&A Recoded':'Auditor_OI','NewItems (excluding 0)':'NewItems'},inplace=True)

dfd['year'] = dfd['fyear'].astype(int)
#dfd['year'] = pd.Categorical(dfd.year)
dfd['year'] = pd.to_datetime(dfd.year, format='%Y')
dfd['cusip'] = pd.Categorical(dfd['cusip'])
dfd = dfd.set_index(['cusip','year'])
dfd['POScore'] = dfd['POST'] * dfd['Score']

#Car0

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore']])

FE = PanelOLS(dfd.Car0, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

Dep. Variable:,Car0,R-squared:,0.0881
Estimator:,PanelOLS,R-squared (Between):,-1.0327
No. Observations:,328,R-squared (Within):,-0.7333
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.9189
Time:,23:28:01,Log-likelihood,457.44
Cov. Estimator:,Clustered,,
,,F-statistic:,3.7527
Entities:,86,P-value,0.0014
Avg Obs:,3.8140,Distribution:,"F(6,233)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,3.8964


In [ ]:
#Investors
#Without CARea
#Adoption Avoidance Group
#Score
#Car1

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore']])

FE = PanelOLS(dfd.Car1, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Car1,R-squared:,0.0903
Estimator:,PanelOLS,R-squared (Between):,-0.5119
No. Observations:,328,R-squared (Within):,-0.8557
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.7419
Time:,19:54:28,Log-likelihood,488.66
Cov. Estimator:,Clustered,,
,,F-statistic:,3.8564
Entities:,86,P-value,0.0011
Avg Obs:,3.8140,Distribution:,"F(6,233)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,10.066


In [ ]:
#Investors
#Without CARea
#Adoption Avoidance Group
#Score
#Car2

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore']])

FE = PanelOLS(dfd.Car2, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Car2,R-squared:,0.0863
Estimator:,PanelOLS,R-squared (Between):,-0.9530
No. Observations:,328,R-squared (Within):,-0.7096
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.8706
Time:,19:54:48,Log-likelihood,456.39
Cov. Estimator:,Clustered,,
,,F-statistic:,3.6680
Entities:,86,P-value,0.0017
Avg Obs:,3.8140,Distribution:,"F(6,233)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,4.0131


In [ ]:
#Investors
#Without CARea
#Adoption Avoidance Group
#Score
#Car3

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore']])

FE = PanelOLS(dfd.Car3, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Car3,R-squared:,0.0890
Estimator:,PanelOLS,R-squared (Between):,-0.4482
No. Observations:,328,R-squared (Within):,-0.8383
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.7032
Time:,19:55:01,Log-likelihood,487.82
Cov. Estimator:,Clustered,,
,,F-statistic:,3.7960
Entities:,86,P-value,0.0013
Avg Obs:,3.8140,Distribution:,"F(6,233)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,19.869


In [ ]:
#Investors
#Without CARea
#Adoption Avoidance Group
#Score
#Scar0

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore']])

FE = PanelOLS(dfd.Scar0, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Scar0,R-squared:,0.0470
Estimator:,PanelOLS,R-squared (Between):,-0.7001
No. Observations:,328,R-squared (Within):,-0.4448
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.5337
Time:,23:28:40,Log-likelihood,-558.83
Cov. Estimator:,Clustered,,
,,F-statistic:,1.9155
Entities:,86,P-value,0.0791
Avg Obs:,3.8140,Distribution:,"F(6,233)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,4.7753


In [ ]:
#Investors
#Without CARea
#Adoption Avoidance Group
#Score
#Scar1

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore']])

FE = PanelOLS(dfd.Scar1, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Scar1,R-squared:,0.0466
Estimator:,PanelOLS,R-squared (Between):,-0.4193
No. Observations:,325,R-squared (Within):,-0.4329
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.4067
Time:,23:29:16,Log-likelihood,-543.49
Cov. Estimator:,Clustered,,
,,F-statistic:,1.8748
Entities:,86,P-value,0.0860
Avg Obs:,3.7791,Distribution:,"F(6,230)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,3.8357


In [ ]:
#Investors
#Without CARea
#Adoption Avoidance Group
#Score
#Scar2

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore']])

FE = PanelOLS(dfd.Scar2, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Scar2,R-squared:,0.0432
Estimator:,PanelOLS,R-squared (Between):,-0.6007
No. Observations:,328,R-squared (Within):,-0.4322
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.4917
Time:,23:29:34,Log-likelihood,-561.76
Cov. Estimator:,Clustered,,
,,F-statistic:,1.7543
Entities:,86,P-value,0.1094
Avg Obs:,3.8140,Distribution:,"F(6,233)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,4.3393


In [ ]:
#Investors
#Without CARea
#Adoption Avoidance Group
#Score
#Scar3

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore']])

FE = PanelOLS(dfd.Scar3, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Scar3,R-squared:,0.0446
Estimator:,PanelOLS,R-squared (Between):,-0.3561
No. Observations:,328,R-squared (Within):,-0.4237
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.4021
Time:,23:29:48,Log-likelihood,-551.79
Cov. Estimator:,Clustered,,
,,F-statistic:,1.8139
Entities:,86,P-value,0.0972
Avg Obs:,3.8140,Distribution:,"F(6,233)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,4.1176


In [ ]:
#Investors
#With CARea
#Adoption Avoidance Group
#Score
#Car0 + Car_e0

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Car_e0']])

FE = PanelOLS(dfd.Car0, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/results.py:85: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")


Dep. Variable:,Car0,R-squared:,0.7956
Estimator:,PanelOLS,R-squared (Between):,0.5506
No. Observations:,328,R-squared (Within):,0.7266
Date:,"Sat, Aug 14 2021",R-squared (Overall):,0.6305
Time:,23:30:39,Log-likelihood,702.70
Cov. Estimator:,Clustered,,
,,F-statistic:,129.01
Entities:,86,P-value,0.0000
Avg Obs:,3.8140,Distribution:,"F(7,232)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,94.130


In [ ]:
#Investors
#With CARea
#Adoption Avoidance Group
#Score
#Car1 + Car_e1

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Car_e1']])

FE = PanelOLS(dfd.Car1, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/results.py:85: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")


Dep. Variable:,Car1,R-squared:,0.7544
Estimator:,PanelOLS,R-squared (Between):,0.7328
No. Observations:,328,R-squared (Within):,0.6865
Date:,"Sat, Aug 14 2021",R-squared (Overall):,0.6932
Time:,23:31:02,Log-likelihood,703.37
Cov. Estimator:,Clustered,,
,,F-statistic:,101.78
Entities:,86,P-value,0.0000
Avg Obs:,3.8140,Distribution:,"F(7,232)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,80.580


In [ ]:
#Investors
#With CARea
#Adoption Avoidance Group
#Score
#Car2 + Car_e2

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Car_e2']])

FE = PanelOLS(dfd.Car2, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/results.py:85: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")


Dep. Variable:,Car2,R-squared:,0.7978
Estimator:,PanelOLS,R-squared (Between):,0.5514
No. Observations:,328,R-squared (Within):,0.7303
Date:,"Sat, Aug 14 2021",R-squared (Overall):,0.6323
Time:,23:31:18,Log-likelihood,703.71
Cov. Estimator:,Clustered,,
,,F-statistic:,130.74
Entities:,86,P-value,0.0000
Avg Obs:,3.8140,Distribution:,"F(7,232)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,93.894


In [ ]:
#Investors
#With CARea
#Adoption Avoidance Group
#Score
#Car3 + Car_e3

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Car_e3']])

FE = PanelOLS(dfd.Car3, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/results.py:85: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")


Dep. Variable:,Car3,R-squared:,0.7451
Estimator:,PanelOLS,R-squared (Between):,0.7437
No. Observations:,328,R-squared (Within):,0.6743
Date:,"Sat, Aug 14 2021",R-squared (Overall):,0.6907
Time:,23:31:35,Log-likelihood,696.71
Cov. Estimator:,Clustered,,
,,F-statistic:,96.888
Entities:,86,P-value,0.0000
Avg Obs:,3.8140,Distribution:,"F(7,232)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,92.487


In [ ]:
#Investors
#With CARea
#Adoption Avoidance Group
#Score
#Scar0 + Scar_e0

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Scar_e0']])

FE = PanelOLS(dfd.Scar0, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/results.py:85: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")


Dep. Variable:,Scar0,R-squared:,0.7841
Estimator:,PanelOLS,R-squared (Between):,0.4686
No. Observations:,328,R-squared (Within):,0.6387
Date:,"Sat, Aug 14 2021",R-squared (Overall):,0.5738
Time:,23:32:09,Log-likelihood,-315.36
Cov. Estimator:,Clustered,,
,,F-statistic:,120.34
Entities:,86,P-value,0.0000
Avg Obs:,3.8140,Distribution:,"F(7,232)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,30.400


In [ ]:
#Investors
#With CARea
#Adoption Avoidance Group
#Score
#Scar1 + Scar_e1

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Scar_e1']])

FE = PanelOLS(dfd.Scar1, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Scar1,R-squared:,0.6959
Estimator:,PanelOLS,R-squared (Between):,0.5417
No. Observations:,325,R-squared (Within):,0.5805
Date:,"Sat, Aug 14 2021",R-squared (Overall):,0.5636
Time:,23:32:26,Log-likelihood,-357.83
Cov. Estimator:,Clustered,,
,,F-statistic:,74.851
Entities:,86,P-value,0.0000
Avg Obs:,3.7791,Distribution:,"F(7,229)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,7.3586


In [ ]:
#Investors
#With CARea
#Adoption Avoidance Group
#Score
#Scar2 + Scar_e2

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Scar_e2']])

FE = PanelOLS(dfd.Scar2, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/results.py:85: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")


Dep. Variable:,Scar2,R-squared:,0.7863
Estimator:,PanelOLS,R-squared (Between):,0.4838
No. Observations:,328,R-squared (Within):,0.6396
Date:,"Sat, Aug 14 2021",R-squared (Overall):,0.5795
Time:,23:32:43,Log-likelihood,-315.93
Cov. Estimator:,Clustered,,
,,F-statistic:,121.94
Entities:,86,P-value,0.0000
Avg Obs:,3.8140,Distribution:,"F(7,232)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,23.899


In [ ]:
#Investors
#With CARea
#Adoption Avoidance Group
#Score
#Scar3 + Scar_e3

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Scar_e3']])

FE = PanelOLS(dfd.Scar3, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Scar3,R-squared:,0.6912
Estimator:,PanelOLS,R-squared (Between):,0.5334
No. Observations:,328,R-squared (Within):,0.5777
Date:,"Sat, Aug 14 2021",R-squared (Overall):,0.5560
Time:,23:32:59,Log-likelihood,-366.57
Cov. Estimator:,Clustered,,
,,F-statistic:,74.180
Entities:,86,P-value,0.0000
Avg Obs:,3.8140,Distribution:,"F(7,232)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,78.036


In [ ]:
#Analyst
#Without CARea
#Adoption Avoidance Group
#Score

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore']])

FE = PanelOLS(dfd.Revision, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Revision,R-squared:,0.0137
Estimator:,PanelOLS,R-squared (Between):,-0.3126
No. Observations:,238,R-squared (Within):,0.0154
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.1223
Time:,23:33:41,Log-likelihood,336.22
Cov. Estimator:,Clustered,,
,,F-statistic:,0.3471
Entities:,86,P-value,0.9107
Avg Obs:,2.7674,Distribution:,"F(6,150)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,0.8668


In [ ]:
#Analyst
#With CARea
#Adoption Avoidance Group
#Score
#Car_e0

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Car_e0']])

FE = PanelOLS(dfd.Revision, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Revision,R-squared:,0.0248
Estimator:,PanelOLS,R-squared (Between):,-0.2252
No. Observations:,238,R-squared (Within):,0.0251
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.0722
Time:,23:34:14,Log-likelihood,337.57
Cov. Estimator:,Clustered,,
,,F-statistic:,0.5415
Entities:,86,P-value,0.8019
Avg Obs:,2.7674,Distribution:,"F(7,149)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,2.5232


In [ ]:
#Analyst
#With CARea
#Adoption Avoidance Group
#Score
#Car_e1

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Car_e1']])

FE = PanelOLS(dfd.Revision, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Revision,R-squared:,0.0222
Estimator:,PanelOLS,R-squared (Between):,-0.2584
No. Observations:,238,R-squared (Within):,0.0236
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.0860
Time:,23:34:30,Log-likelihood,337.25
Cov. Estimator:,Clustered,,
,,F-statistic:,0.4830
Entities:,86,P-value,0.8458
Avg Obs:,2.7674,Distribution:,"F(7,149)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,-2.1939


In [ ]:
#Analyst
#With CARea
#Adoption Avoidance Group
#Score
#Car_e2

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Car_e2']])

FE = PanelOLS(dfd.Revision, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Revision,R-squared:,0.0258
Estimator:,PanelOLS,R-squared (Between):,-0.2266
No. Observations:,238,R-squared (Within):,0.0255
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.0725
Time:,23:34:46,Log-likelihood,337.69
Cov. Estimator:,Clustered,,
,,F-statistic:,0.5632
Entities:,86,P-value,0.7848
Avg Obs:,2.7674,Distribution:,"F(7,149)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,2.8073


In [ ]:
#Analyst
#With CARea
#Adoption Avoidance Group
#Score
#Car_e3

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Car_e3']])

FE = PanelOLS(dfd.Revision, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Revision,R-squared:,0.0224
Estimator:,PanelOLS,R-squared (Between):,-0.2598
No. Observations:,238,R-squared (Within):,0.0236
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.0869
Time:,23:34:58,Log-likelihood,337.28
Cov. Estimator:,Clustered,,
,,F-statistic:,0.4881
Entities:,86,P-value,0.8421
Avg Obs:,2.7674,Distribution:,"F(7,149)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,-2.0067


In [ ]:
#Analyst
#With CARea
#Adoption Avoidance Group
#Score
#Scar_e0

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Scar_e0']])

FE = PanelOLS(dfd.Revision, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Revision,R-squared:,0.0139
Estimator:,PanelOLS,R-squared (Between):,-0.3031
No. Observations:,238,R-squared (Within):,0.0140
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.1190
Time:,23:35:12,Log-likelihood,336.24
Cov. Estimator:,Clustered,,
,,F-statistic:,0.2990
Entities:,86,P-value,0.9533
Avg Obs:,2.7674,Distribution:,"F(7,149)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,1.7718


In [ ]:
#Analyst
#With CARea
#Adoption Avoidance Group
#Score
#Scar_e1

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Scar_e1']])

FE = PanelOLS(dfd.Revision, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Revision,R-squared:,0.0139
Estimator:,PanelOLS,R-squared (Between):,-0.3320
No. Observations:,235,R-squared (Within):,0.0149
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.1274
Time:,23:35:29,Log-likelihood,330.80
Cov. Estimator:,Clustered,,
,,F-statistic:,0.2938
Entities:,86,P-value,0.9554
Avg Obs:,2.7326,Distribution:,"F(7,146)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,3.5131


In [ ]:
#Analyst
#With CARea
#Adoption Avoidance Group
#Score
#Scar_e2

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Scar_e2']])

FE = PanelOLS(dfd.Revision, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Revision,R-squared:,0.0141
Estimator:,PanelOLS,R-squared (Between):,-0.2994
No. Observations:,238,R-squared (Within):,0.0131
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.1179
Time:,23:35:43,Log-likelihood,336.27
Cov. Estimator:,Clustered,,
,,F-statistic:,0.3039
Entities:,86,P-value,0.9512
Avg Obs:,2.7674,Distribution:,"F(7,149)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,2.1642


In [ ]:
#Analyst
#With CARea
#Adoption Avoidance Group
#Score
#Scar_e3

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Scar_e3']])

FE = PanelOLS(dfd.Revision, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,Revision,R-squared:,0.0137
Estimator:,PanelOLS,R-squared (Between):,-0.3154
No. Observations:,238,R-squared (Within):,0.0159
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.1233
Time:,23:35:56,Log-likelihood,336.23
Cov. Estimator:,Clustered,,
,,F-statistic:,0.2960
Entities:,86,P-value,0.9546
Avg Obs:,2.7674,Distribution:,"F(7,149)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,2.2363


In [ ]:

import pandas as pd
from linearmodels.panel import PanelOLS
from statsmodels.tools.tools import add_constant

import numpy as np

df = pd.read_excel('/content/Non-Adoption Group Data August 7, 2021.xlsx') 

#Alternative for fixed effect time-series
dfd = df[['Revision','cusip','fyear','Car0','Car1','Car2','Car3','Scar0','Scar1','Scar2','Scar3','Car_e0','Car_e1','Car_e2','Car_e3','Scar_e0', 'Scar_e1', 'Scar_e2','Scar_e3','Other information MD&A Recoded','Score','Raw Score','Size','Filelate','NewItems (excluding 0)','POST']]

dfd.rename(columns={'Other information MD&A Recoded':'Auditor_OI','NewItems (excluding 0)':'NewItems'},inplace=True)

dfd['year'] = dfd['fyear'].astype(int)
#dfd['year'] = pd.Categorical(dfd.year)
dfd['year'] = pd.to_datetime(dfd.year, format='%Y')
dfd['cusip'] = pd.Categorical(dfd['cusip'])
dfd = dfd.set_index(['cusip','year'])
dfd['POScore'] = dfd['POST'] * dfd['Score']

#Car0
#Score

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore']])

FE = PanelOLS(dfd.Car0, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

Dep. Variable:,Car0,R-squared:,0.3085
Estimator:,PanelOLS,R-squared (Between):,-3.4572
No. Observations:,77,R-squared (Within):,0.3021
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-3.0156
Time:,23:43:14,Log-likelihood,158.11
Cov. Estimator:,Clustered,,
,,F-statistic:,4.1929
Entities:,22,P-value,0.0031
Avg Obs:,3.5000,Distribution:,"F(5,47)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,2.1874


In [ ]:
#Investors
#Without CARea
#Non-Adoption Group
#Score
#Car1

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore']])

FE = PanelOLS(dfd.Car1, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/model.py:1798: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Filelate

  AbsorbingEffectWarning,


Dep. Variable:,Car1,R-squared:,0.2637
Estimator:,PanelOLS,R-squared (Between):,-4.3176
No. Observations:,77,R-squared (Within):,0.2517
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-3.3113
Time:,23:44:33,Log-likelihood,165.45
Cov. Estimator:,Clustered,,
,,F-statistic:,3.3671
Entities:,22,P-value,0.0111
Avg Obs:,3.5000,Distribution:,"F(5,47)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,2.1987


In [ ]:
#Investors
#Without CARea
#Non-Adoption Group
#Score
#Car2

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore']])

FE = PanelOLS(dfd.Car2, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/model.py:1798: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Filelate

  AbsorbingEffectWarning,


Dep. Variable:,Car2,R-squared:,0.3368
Estimator:,PanelOLS,R-squared (Between):,-3.6869
No. Observations:,77,R-squared (Within):,0.3242
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-3.1843
Time:,23:44:57,Log-likelihood,162.23
Cov. Estimator:,Clustered,,
,,F-statistic:,4.7731
Entities:,22,P-value,0.0013
Avg Obs:,3.5000,Distribution:,"F(5,47)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,2.4748


In [ ]:
#Investors
#Without CARea
#Non-Adoption Group
#Score
#Car3

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore']])

FE = PanelOLS(dfd.Car3, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/model.py:1798: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Filelate

  AbsorbingEffectWarning,


Dep. Variable:,Car3,R-squared:,0.2926
Estimator:,PanelOLS,R-squared (Between):,-5.4943
No. Observations:,77,R-squared (Within):,0.2774
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-3.8753
Time:,23:45:09,Log-likelihood,168.58
Cov. Estimator:,Clustered,,
,,F-statistic:,3.8877
Entities:,22,P-value,0.0050
Avg Obs:,3.5000,Distribution:,"F(5,47)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,2.2100


In [ ]:
#Investors
#Without CARea
#Non-Adoption Group
#Score
#Scar0

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore']])

FE = PanelOLS(dfd.Scar0, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/model.py:1798: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Filelate

  AbsorbingEffectWarning,


Dep. Variable:,Scar0,R-squared:,0.1362
Estimator:,PanelOLS,R-squared (Between):,-2.1054
No. Observations:,77,R-squared (Within):,0.0155
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.7262
Time:,23:45:26,Log-likelihood,-107.09
Cov. Estimator:,Clustered,,
,,F-statistic:,1.4817
Entities:,22,P-value,0.2138
Avg Obs:,3.5000,Distribution:,"F(5,47)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,1.6495


In [ ]:
#Investors
#Without CARea
#Non-Adoption Group
#Score
#Scar1

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore']])

FE = PanelOLS(dfd.Scar1, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/model.py:1798: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Filelate

  AbsorbingEffectWarning,


Dep. Variable:,Scar1,R-squared:,0.1136
Estimator:,PanelOLS,R-squared (Between):,-3.2732
No. Observations:,77,R-squared (Within):,0.0428
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.9398
Time:,23:45:42,Log-likelihood,-106.08
Cov. Estimator:,Clustered,,
,,F-statistic:,1.2049
Entities:,22,P-value,0.3216
Avg Obs:,3.5000,Distribution:,"F(5,47)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,1.5168


In [ ]:
#Investors
#Without CARea
#Non-Adoption Group
#Score
#Scar2

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore']])

FE = PanelOLS(dfd.Scar2, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/model.py:1798: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Filelate

  AbsorbingEffectWarning,


Dep. Variable:,Scar2,R-squared:,0.1504
Estimator:,PanelOLS,R-squared (Between):,-2.0522
No. Observations:,77,R-squared (Within):,-0.0183
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-0.8380
Time:,23:46:08,Log-likelihood,-103.99
Cov. Estimator:,Clustered,,
,,F-statistic:,1.6634
Entities:,22,P-value,0.1621
Avg Obs:,3.5000,Distribution:,"F(5,47)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,1.6167


In [ ]:
#Investors
#Without CARea
#Non-Adoption Group
#Score
#Scar3

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore']])

FE = PanelOLS(dfd.Scar3, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/model.py:1798: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Filelate

  AbsorbingEffectWarning,


Dep. Variable:,Scar3,R-squared:,0.1198
Estimator:,PanelOLS,R-squared (Between):,-4.2559
No. Observations:,77,R-squared (Within):,0.0170
Date:,"Sat, Aug 14 2021",R-squared (Overall):,-1.3461
Time:,23:46:23,Log-likelihood,-105.00
Cov. Estimator:,Clustered,,
,,F-statistic:,1.2795
Entities:,22,P-value,0.2886
Avg Obs:,3.5000,Distribution:,"F(5,47)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,2.5324


In [ ]:
#Investors
#With CARea
#Non-Adoption Group
#Score
#Car0 + Car_e0

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Car_e0']])

FE = PanelOLS(dfd.Car0, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/model.py:1798: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Filelate

  AbsorbingEffectWarning,


Dep. Variable:,Car0,R-squared:,0.4599
Estimator:,PanelOLS,R-squared (Between):,-0.7417
No. Observations:,77,R-squared (Within):,0.4626
Date:,"Sun, Aug 15 2021",R-squared (Overall):,-0.6802
Time:,01:06:18,Log-likelihood,167.62
Cov. Estimator:,Clustered,,
,,F-statistic:,6.5281
Entities:,22,P-value,0.0000
Avg Obs:,3.5000,Distribution:,"F(6,46)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,4.5573


In [ ]:
#Investors
#With CARea
#Non-Adoption Group
#Score
#Car1 + Car_e1

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Car_e1']])

FE = PanelOLS(dfd.Car1, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/model.py:1798: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Filelate

  AbsorbingEffectWarning,
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/results.py:85: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")


Dep. Variable:,Car1,R-squared:,0.5324
Estimator:,PanelOLS,R-squared (Between):,-1.1643
No. Observations:,77,R-squared (Within):,0.5367
Date:,"Sun, Aug 15 2021",R-squared (Overall):,-0.8324
Time:,01:06:59,Log-likelihood,182.93
Cov. Estimator:,Clustered,,
,,F-statistic:,8.7295
Entities:,22,P-value,0.0000
Avg Obs:,3.5000,Distribution:,"F(6,46)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,3.6826


In [ ]:
#Investors
#With CARea
#Non-Adoption Group
#Score
#Car2 + Car_e2

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Car_e2']])

FE = PanelOLS(dfd.Car2, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/model.py:1798: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Filelate

  AbsorbingEffectWarning,


Dep. Variable:,Car2,R-squared:,0.4527
Estimator:,PanelOLS,R-squared (Between):,-1.1154
No. Observations:,77,R-squared (Within):,0.4530
Date:,"Sun, Aug 15 2021",R-squared (Overall):,-0.9821
Time:,01:07:16,Log-likelihood,169.62
Cov. Estimator:,Clustered,,
,,F-statistic:,6.3418
Entities:,22,P-value,0.0001
Avg Obs:,3.5000,Distribution:,"F(6,46)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,4.4339


In [ ]:
#Investors
#With CARea
#Non-Adoption Group
#Score
#Car3 + Car_e3

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Car_e3']])

FE = PanelOLS(dfd.Car3, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/model.py:1798: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Filelate

  AbsorbingEffectWarning,
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/results.py:85: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")


Dep. Variable:,Car3,R-squared:,0.5172
Estimator:,PanelOLS,R-squared (Between):,-1.9227
No. Observations:,77,R-squared (Within):,0.5234
Date:,"Sun, Aug 15 2021",R-squared (Overall):,-1.2395
Time:,01:07:40,Log-likelihood,183.29
Cov. Estimator:,Clustered,,
,,F-statistic:,8.2133
Entities:,22,P-value,0.0000
Avg Obs:,3.5000,Distribution:,"F(6,46)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,3.5047


In [ ]:
#Investors
#With CARea
#Non-Adoption Group
#Score
#Scar0 + Scar_e0

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Scar_e0']])

FE = PanelOLS(dfd.Scar0, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/model.py:1798: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Filelate

  AbsorbingEffectWarning,


Dep. Variable:,Scar0,R-squared:,0.2884
Estimator:,PanelOLS,R-squared (Between):,-0.6154
No. Observations:,77,R-squared (Within):,0.2362
Date:,"Sun, Aug 15 2021",R-squared (Overall):,-0.0750
Time:,01:08:05,Log-likelihood,-99.626
Cov. Estimator:,Clustered,,
,,F-statistic:,3.1076
Entities:,22,P-value,0.0122
Avg Obs:,3.5000,Distribution:,"F(6,46)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,2.9014


In [ ]:
#Investors
#With CARea
#Non-Adoption Group
#Score
#Scar1 + Scar_e1

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Scar_e1']])

FE = PanelOLS(dfd.Scar1, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/model.py:1798: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Filelate

  AbsorbingEffectWarning,
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/results.py:85: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")


Dep. Variable:,Scar1,R-squared:,0.3480
Estimator:,PanelOLS,R-squared (Between):,-1.6127
No. Observations:,77,R-squared (Within):,0.3591
Date:,"Sun, Aug 15 2021",R-squared (Overall):,-0.2551
Time:,01:08:51,Log-likelihood,-94.261
Cov. Estimator:,Clustered,,
,,F-statistic:,4.0915
Entities:,22,P-value,0.0023
Avg Obs:,3.5000,Distribution:,"F(6,46)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,1.9659


In [ ]:
#Investors
#With CARea
#Non-Adoption Group
#Score
#Scar2 + Scar_e2

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Scar_e2']])

FE = PanelOLS(dfd.Scar2, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/model.py:1798: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Filelate

  AbsorbingEffectWarning,


Dep. Variable:,Scar2,R-squared:,0.2781
Estimator:,PanelOLS,R-squared (Between):,-0.6196
No. Observations:,77,R-squared (Within):,0.1954
Date:,"Sun, Aug 15 2021",R-squared (Overall):,-0.1403
Time:,01:09:08,Log-likelihood,-97.713
Cov. Estimator:,Clustered,,
,,F-statistic:,2.9534
Entities:,22,P-value,0.0160
Avg Obs:,3.5000,Distribution:,"F(6,46)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,1.4663


In [ ]:
#Investors
#With CARea
#Non-Adoption Group
#Score
#Scar3 + Scar_e3

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Scar_e3']])

FE = PanelOLS(dfd.Scar3, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/model.py:1798: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Filelate

  AbsorbingEffectWarning,


Dep. Variable:,Scar3,R-squared:,0.3220
Estimator:,PanelOLS,R-squared (Between):,-2.1988
No. Observations:,77,R-squared (Within):,0.3247
Date:,"Sun, Aug 15 2021",R-squared (Overall):,-0.5176
Time:,01:09:29,Log-likelihood,-94.955
Cov. Estimator:,Clustered,,
,,F-statistic:,3.6403
Entities:,22,P-value,0.0049
Avg Obs:,3.5000,Distribution:,"F(6,46)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,2.6642


In [ ]:
#Analyst
#Without CARea
#Non-Adoption Group
#Score

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore']])

FE = PanelOLS(dfd.Revision, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/model.py:1798: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Filelate

  AbsorbingEffectWarning,


Dep. Variable:,Revision,R-squared:,0.4136
Estimator:,PanelOLS,R-squared (Between):,-7.6594
No. Observations:,62,R-squared (Within):,0.3993
Date:,"Sun, Aug 15 2021",R-squared (Overall):,-5.3500
Time:,01:27:02,Log-likelihood,118.76
Cov. Estimator:,Clustered,,
,,F-statistic:,4.7971
Entities:,22,P-value,0.0020
Avg Obs:,2.8182,Distribution:,"F(5,34)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,29.095


In [ ]:
#Analyst
#With CARea
#Non-Adoption Group
#Score
#Car_e0

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Car_e0']])

FE = PanelOLS(dfd.Revision, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/model.py:1798: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Filelate

  AbsorbingEffectWarning,


Dep. Variable:,Revision,R-squared:,0.5257
Estimator:,PanelOLS,R-squared (Between):,-3.6427
No. Observations:,61,R-squared (Within):,0.4553
Date:,"Sun, Aug 15 2021",R-squared (Overall):,-2.5369
Time:,01:28:21,Log-likelihood,122.83
Cov. Estimator:,Clustered,,
,,F-statistic:,5.9111
Entities:,22,P-value,0.0003
Avg Obs:,2.7727,Distribution:,"F(6,32)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,1.2757


In [ ]:
#Analyst
#With CARea
#Non-Adoption Group
#Score
#Car_e1

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Car_e1']])

FE = PanelOLS(dfd.Revision, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/model.py:1798: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Filelate

  AbsorbingEffectWarning,


Dep. Variable:,Revision,R-squared:,0.5364
Estimator:,PanelOLS,R-squared (Between):,-4.6334
No. Observations:,61,R-squared (Within):,0.3912
Date:,"Sun, Aug 15 2021",R-squared (Overall):,-3.3764
Time:,01:28:32,Log-likelihood,123.53
Cov. Estimator:,Clustered,,
,,F-statistic:,6.1708
Entities:,22,P-value,0.0002
Avg Obs:,2.7727,Distribution:,"F(6,32)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,1.2220


In [ ]:
#Analyst
#With CARea
#Non-Adoption Group
#Score
#Car_e2

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Car_e2']])

FE = PanelOLS(dfd.Revision, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/model.py:1798: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Filelate

  AbsorbingEffectWarning,


Dep. Variable:,Revision,R-squared:,0.5088
Estimator:,PanelOLS,R-squared (Between):,-3.6217
No. Observations:,61,R-squared (Within):,0.4508
Date:,"Sun, Aug 15 2021",R-squared (Overall):,-2.5271
Time:,01:28:45,Log-likelihood,121.76
Cov. Estimator:,Clustered,,
,,F-statistic:,5.5248
Entities:,22,P-value,0.0005
Avg Obs:,2.7727,Distribution:,"F(6,32)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,1.2302


In [ ]:
#Analyst
#With CARea
#Non-Adoption Group
#Score
#Car_e3

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Car_e3']])

FE = PanelOLS(dfd.Revision, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/model.py:1798: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Filelate

  AbsorbingEffectWarning,


Dep. Variable:,Revision,R-squared:,0.5260
Estimator:,PanelOLS,R-squared (Between):,-4.3036
No. Observations:,61,R-squared (Within):,0.3957
Date:,"Sun, Aug 15 2021",R-squared (Overall):,-3.1427
Time:,01:28:59,Log-likelihood,122.85
Cov. Estimator:,Clustered,,
,,F-statistic:,5.9187
Entities:,22,P-value,0.0003
Avg Obs:,2.7727,Distribution:,"F(6,32)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,1.1775


In [ ]:
#Analyst
#With CARea
#Non-Adoption Group
#Score
#Scar_e0

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Scar_e0']])

FE = PanelOLS(dfd.Revision, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/model.py:1798: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Filelate

  AbsorbingEffectWarning,


Dep. Variable:,Revision,R-squared:,0.4251
Estimator:,PanelOLS,R-squared (Between):,-7.3685
No. Observations:,61,R-squared (Within):,0.3588
Date:,"Sun, Aug 15 2021",R-squared (Overall):,-5.1899
Time:,01:29:14,Log-likelihood,116.96
Cov. Estimator:,Clustered,,
,,F-statistic:,3.9440
Entities:,22,P-value,0.0046
Avg Obs:,2.7727,Distribution:,"F(6,32)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,1.2995


In [ ]:
#Analyst
#With CARea
#Non-Adoption Group
#Score
#Scar_e1

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Scar_e1']])

FE = PanelOLS(dfd.Revision, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/model.py:1798: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Filelate

  AbsorbingEffectWarning,


Dep. Variable:,Revision,R-squared:,0.4258
Estimator:,PanelOLS,R-squared (Between):,-7.5908
No. Observations:,61,R-squared (Within):,0.3442
Date:,"Sun, Aug 15 2021",R-squared (Overall):,-5.3838
Time:,01:29:28,Log-likelihood,117.00
Cov. Estimator:,Clustered,,
,,F-statistic:,3.9556
Entities:,22,P-value,0.0045
Avg Obs:,2.7727,Distribution:,"F(6,32)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,1.2552


In [ ]:
#Analyst
#With CARea
#Non-Adoption Group
#Score
#Scar_e2

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Scar_e2']])

FE = PanelOLS(dfd.Revision, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/model.py:1798: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Filelate

  AbsorbingEffectWarning,


Dep. Variable:,Revision,R-squared:,0.4210
Estimator:,PanelOLS,R-squared (Between):,-7.3974
No. Observations:,61,R-squared (Within):,0.3612
Date:,"Sun, Aug 15 2021",R-squared (Overall):,-5.1979
Time:,01:29:40,Log-likelihood,116.75
Cov. Estimator:,Clustered,,
,,F-statistic:,3.8782
Entities:,22,P-value,0.0051
Avg Obs:,2.7727,Distribution:,"F(6,32)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,1.6637


In [ ]:
#Analyst
#With CARea
#Non-Adoption Group
#Score
#Scar_e3

exog = add_constant(dfd[['Score','Size', 'NewItems','Filelate', 'POST', 'POScore','Scar_e3']])

FE = PanelOLS(dfd.Revision, exog,
              entity_effects = True,
              time_effects= True,
              check_rank=False,
              drop_absorbed=True)
              
# Result
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
             cluster_time=True
             )

result

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/usr/local/lib/python3.7/dist-packages/linearmodels/panel/model.py:1798: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Filelate

  AbsorbingEffectWarning,


Dep. Variable:,Revision,R-squared:,0.4220
Estimator:,PanelOLS,R-squared (Between):,-7.5355
No. Observations:,61,R-squared (Within):,0.3478
Date:,"Sun, Aug 15 2021",R-squared (Overall):,-5.3281
Time:,01:29:49,Log-likelihood,116.80
Cov. Estimator:,Clustered,,
,,F-statistic:,3.8942
Entities:,22,P-value,0.0050
Avg Obs:,2.7727,Distribution:,"F(6,32)"
Min Obs:,0.0000,,
Max Obs:,4.0000,F-statistic (robust):,1.4687


In [ ]:
dfd.index

MultiIndex([(  'AEM', '2016-01-01'),
            (  'AEM', '2017-01-01'),
            (  'AEM', '2018-01-01'),
            (  'AEM', '2019-01-01'),
            (  'BHC', '2016-01-01'),
            (  'BHC', '2017-01-01'),
            (  'BHC', '2018-01-01'),
            (  'BHC', '2019-01-01'),
            (  'CAE', '2015-01-01'),
            (  'CAE', '2016-01-01'),
            (  'CAE', '2017-01-01'),
            (  'CAE', '2018-01-01'),
            (  'CAE', '2019-01-01'),
            (   'CP', '2016-01-01'),
            (   'CP', '2017-01-01'),
            (   'CP', '2018-01-01'),
            (   'CP', '2019-01-01'),
            (   'CP', '2020-01-01'),
            (  'CLS', '2016-01-01'),
            (  'CLS', '2017-01-01'),
            (  'CLS', '2018-01-01'),
            (  'CLS', '2019-01-01'),
            (  'CLS', '2020-01-01'),
            (  'CVE', '2016-01-01'),
            (  'CVE', '2017-01-01'),
            (  'CVE', '2018-01-01'),
            (  'CVE', '2019-01-01'),
 

In [ ]:
exog

const  Score       Size  ...  NewItems  Auditor_OI  AuScore
tic year                           ...                               
AEM 2016    1.0    NaN   8.868969  ...       153           0      NaN
    2017    1.0   0.84   8.970254  ...       157           0      0.0
    2018    1.0   0.83   8.968631  ...       158           0      0.0
    2019    1.0   0.84   9.081357  ...       147           0      0.0
BHC 2016    1.0    NaN  10.681183  ...       194           0      NaN
...         ...    ...        ...  ...       ...         ...      ...
WCN 2016    1.0    NaN   9.323053  ...       169           0      NaN
    2017    1.0   0.83   9.393885  ...       165           0      0.0
    2018    1.0   0.82   9.443619  ...       162           0      0.0
    2019    1.0   0.80   9.527899  ...       156           0      0.0
    2020    1.0   0.81   9.546267  ...       152           0      0.0

[93 rows x 8 columns]

In [ ]:
pip install statsmodels


  Using cached https://files.pythonhosted.org/packages/da/69/8eef30a6237c54f3c0b524140e2975f4b1eea3489b45eb3339574fc8acee/statsmodels-0.12.2-cp37-cp37m-manylinux1_x86_64.whl


In [ ]:
pip install pandas 

In [ ]:
len(dfd)


93

In [ ]:
# For Claire
# spreg has a seperate api from pysal now
import spreg
#import pysal as ps

fe1_d = pd.get_dummies(dfd['fyear'], 'd_year')
exog = sm.add_constant(dfd[['Score','Size','NewItems']])
xone = np.hstack((exog, fe1_d.values))
m1_dummy = spreg.BaseOLS(dfd['Car_e1'].values, xone)
m1_dummy.betas

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])